## Meta matching v1.0
This jupyter notebook demonstrates you how to load and use meta-matching algorthm. In this demonstration, we performed Meta-matching (DNN) stacking with 100 example subjects.

### Step 0. Setup
Please modify the `path_repo` below to your repo position:

In [1]:
# initialization and random seed set

import os
import scipy
import torch
import random
import sklearn
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from CBIG_model_pytorch import dnn_4l, dnn_5l
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings("ignore")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Please modify the gpu number here if you want to use different gpu. If the gpu you assigned not availiable, it will assign to cpu

### Step 0-1. Meta matching - meta train

### Step 1. load data

In [2]:
subejct_df = pd.read_csv('for_reproduce/MMP_HCP_753_Age_list.csv', index_col = 0)

In [4]:
#unrestricted_sophiafrangou_7_11_2021_22_0_0
phenotype = pd.read_csv('for_reproduce/unrestricted_sophiafrangou_7_11_2021_22_0_0.csv', index_col = 0)
phenotype.head()

,Release,Acquisition,Gender,Age,3T_Full_MR_Compl,T1_Count,T2_Count,3T_RS-fMRI_Count,3T_RS-fMRI_PctCompl,3T_Full_Task_fMRI,...,Noise_Comp,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
Subject,,,,,,,,,,,,,,,,,,,,,
100004,S900,Q06,M,22-25,False,0,0,0,0.0,False,...,5.2,101.12,86.45,2.0,45.9,107.17,105.31,1.80,0.0,1.80
100206,S900,Q11,M,26-30,True,1,1,4,100.0,True,...,6.0,108.79,97.19,1.0,49.7,72.63,72.03,1.84,0.0,1.84
100307,Q1,Q01,F,26-30,True,1,1,4,100.0,True,...,3.6,101.12,86.45,0.0,38.6,71.69,71.76,1.76,0.0,1.76
100408,Q3,Q03,M,31-35,True,1,1,4,100.0,True,...,2.0,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68
100610,S900,Q08,M,26-30,True,2,1,4,100.0,True,...,2.0,122.25,110.45,0.0,38.6,84.84,85.31,1.92,1.0,1.88


In [6]:
pheno_list = ['PicSeq_Unadj', 'CardSort_Unadj', 'Flanker_Unadj',
 'PMAT24_A_CR', 'ReadEng_Unadj', 'PicVocab_Unadj', 'ProcSpeed_Unadj',
 'DDisc_AUC_40K', 'VSPLOT_TC', 'SCPT_SEN', 'SCPT_SPEC',
 'IWRD_TOT', 'ListSort_Unadj', 'MMSE_Score', 'PSQI_Score', 'Endurance_Unadj',
 'GaitSpeed_Comp', 'Dexterity_Unadj', 'Strength_Unadj', 'Odor_Unadj', 'PainInterf_Tscore',
 'Taste_Unadj', 'Mars_Final', 'Emotion_Task_Face_Acc',
 'Language_Task_Math_Avg_Difficulty_Level', 'Language_Task_Story_Avg_Difficulty_Level',
 'Relational_Task_Acc', 'Social_Task_Perc_Random',
 'Social_Task_Perc_TOM', 'WM_Task_Acc', 'NEOFAC_A', 'NEOFAC_O',
 'NEOFAC_C', 'NEOFAC_N', 'NEOFAC_E', 'ER40_CR', 'ER40ANG', 'ER40FEAR', 'ER40HAP', 'ER40NOE',
 'ER40SAD', 'AngAffect_Unadj', 'AngHostil_Unadj', 'AngAggr_Unadj', 'FearAffect_Unadj',
 'FearSomat_Unadj', 'Sadness_Unadj', 'LifeSatisf_Unadj',
 'MeanPurp_Unadj', 'PosAffect_Unadj', 'Friendship_Unadj', 'Loneliness_Unadj',
 'PercHostil_Unadj', 'PercReject_Unadj', 'EmotSupp_Unadj', 'InstruSupp_Unadj',
 'PercStress_Unadj', 'SelfEff_Unadj']

In [7]:
pheno_58 = phenotype.loc[:, pheno_list]
pheno_58

,PicSeq_Unadj,CardSort_Unadj,Flanker_Unadj,PMAT24_A_CR,ReadEng_Unadj,PicVocab_Unadj,ProcSpeed_Unadj,DDisc_AUC_40K,VSPLOT_TC,SCPT_SEN,...,MeanPurp_Unadj,PosAffect_Unadj,Friendship_Unadj,Loneliness_Unadj,PercHostil_Unadj,PercReject_Unadj,EmotSupp_Unadj,InstruSupp_Unadj,PercStress_Unadj,SelfEff_Unadj
Subject,,,,,,,,,,,,,,,,,,,,,
100004,118.70,103.17,121.97,19.0,109.9589,111.1118,92.72,0.067448,9.0,0.8833,...,40.1,36.2,27.6,73.0,60.0,70.4,29.3,34.8,55.4,39.6
100206,125.07,119.14,130.42,20.0,113.5460,119.8914,138.72,0.050000,12.0,0.9667,...,50.0,48.4,45.7,63.8,63.8,64.8,43.9,46.0,57.8,44.0
100307,110.61,123.75,112.56,17.0,111.0700,107.2000,122.65,0.311459,9.0,0.9833,...,42.4,46.2,62.0,53.7,52.7,54.1,45.0,52.3,57.9,37.5
100408,125.71,111.14,121.18,7.0,131.8100,134.2400,107.08,0.421354,17.0,0.8667,...,51.9,48.4,52.9,51.9,52.2,50.1,50.1,51.1,46.8,56.4
100610,109.04,129.84,126.53,23.0,141.3166,140.8151,111.11,0.868750,18.0,1.0000,...,46.8,55.1,44.9,53.5,38.5,54.1,56.8,48.4,37.8,53.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992774,89.89,126.37,124.64,14.0,108.6900,106.0900,114.31,0.019531,9.0,0.9333,...,50.0,48.4,47.5,49.6,52.5,50.6,51.8,46.7,46.7,54.7
993675,108.89,120.35,116.43,21.0,129.4080,135.6653,113.40,0.938281,21.0,0.9833,...,59.8,46.2,50.7,53.7,53.9,54.1,44.8,45.9,49.8,45.9
994273,87.10,112.17,123.22,20.0,141.3166,122.3772,83.25,0.529427,13.0,0.9500,...,38.8,46.5,55.2,53.7,62.5,40.7,51.8,62.9,60.4,48.0


In [8]:
subject_list = subejct_df.index.tolist()
pheno_58 = pheno_58.loc[subject_list, :]
pheno_58.shape

(753, 58)

In [9]:
age = subejct_df[['Age']]

In [10]:
pheno_with_age = pd.concat([pheno_58, age], axis = 1)
pheno_with_age

,PicSeq_Unadj,CardSort_Unadj,Flanker_Unadj,PMAT24_A_CR,ReadEng_Unadj,PicVocab_Unadj,ProcSpeed_Unadj,DDisc_AUC_40K,VSPLOT_TC,SCPT_SEN,...,PosAffect_Unadj,Friendship_Unadj,Loneliness_Unadj,PercHostil_Unadj,PercReject_Unadj,EmotSupp_Unadj,InstruSupp_Unadj,PercStress_Unadj,SelfEff_Unadj,Age
Subject,,,,,,,,,,,,,,,,,,,,,
100206,125.07,119.14,130.42,20.0,113.5460,119.89140,138.72,0.050000,12.0,0.9667,...,48.4,45.7,63.8,63.8,64.8,43.9,46.0,57.8,44.0,27
100307,110.61,123.75,112.56,17.0,111.0700,107.20000,122.65,0.311459,9.0,0.9833,...,46.2,62.0,53.7,52.7,54.1,45.0,52.3,57.9,37.5,27
100408,125.71,111.14,121.18,7.0,131.8100,134.24000,107.08,0.421354,17.0,0.8667,...,48.4,52.9,51.9,52.2,50.1,50.1,51.1,46.8,56.4,33
101006,84.68,105.19,101.85,11.0,113.5374,95.42348,90.59,0.783073,20.0,0.9833,...,42.0,57.1,50.3,46.1,46.5,62.5,43.3,48.8,38.8,35
101107,105.60,119.76,107.04,14.0,119.2434,119.84590,112.27,0.584375,16.0,0.9167,...,59.2,66.5,52.0,63.8,43.7,62.5,32.6,51.9,68.4,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992673,96.55,94.96,94.87,21.0,117.9919,118.44630,104.06,0.791667,11.0,0.8833,...,50.2,49.8,50.3,47.8,40.7,62.5,46.7,41.0,49.3,33
992774,89.89,126.37,124.64,14.0,108.6900,106.09000,114.31,0.019531,9.0,0.9333,...,48.4,47.5,49.6,52.5,50.6,51.8,46.7,46.7,54.7,35
993675,108.89,120.35,116.43,21.0,129.4080,135.66530,113.40,0.938281,21.0,0.9833,...,46.2,50.7,53.7,53.9,54.1,44.8,45.9,49.8,45.9,29


In [11]:
"""cort_thick = np.load('for_reproduce/smri/cortical_thick.npy')
cort_area = np.load('for_reproduce/smri/cortical_area.npy')
cort_vol = np.load('for_reproduce/smri/cortical_vol.npy')

tbss_ad = np.load('for_reproduce/dmri/tbss/tbss_ad.npy')
tbss_fa = np.load('for_reproduce/dmri/tbss/tbss_fa.npy')
tbss_icvf = np.load('for_reproduce/dmri/tbss/tbss_icvf.npy')
tbss_isovf = np.load('for_reproduce/dmri/tbss/tbss_isovf.npy')
tbss_md = np.load('for_reproduce/dmri/tbss/tbss_md.npy')
tbss_od = np.load('for_reproduce/dmri/tbss/tbss_od.npy')
tbss_rd = np.load('for_reproduce/dmri/tbss/tbss_rd.npy')

tractography_ad = np.load('for_reproduce/dmri/tracto/tracto_ad.npy')
tractography_fa = np.load('for_reproduce/dmri/tracto/tracto_fa.npy')
tractography_icvf = np.load('for_reproduce/dmri/tracto/tracto_icvf.npy')
tractography_isovf = np.load('for_reproduce/dmri/tracto/tracto_isovf.npy')
tractography_md = np.load('for_reproduce/dmri/tracto/tracto_md.npy')
tractography_od = np.load('for_reproduce/dmri/tracto/tracto_od.npy')
tractography_rd = np.load('for_reproduce/dmri/tracto/tracto_rd.npy')
tractography_fss = np.load('for_reproduce/dmri/tracto/tracto_fss.npy')
tractography_fssl = np.load('for_reproduce/dmri/tracto/tracto_fssl.npy')

f_gamb = np.load('for_reproduce/fmri/f_gamb.npy')
f_lang = np.load('for_reproduce/fmri/f_lang.npy')
f_motor = np.load('for_reproduce/fmri/f_motor.npy')
f_rs = np.load('for_reproduce/fmri/f_rs.npy')
f_social = np.load('for_reproduce/fmri/f_social.npy')
f_wm = np.load('for_reproduce/fmri/f_wm.npy')"""

"cort_thick = np.load('for_reproduce/smri/cortical_thick.npy')\ncort_area = np.load('for_reproduce/smri/cortical_area.npy')\ncort_vol = np.load('for_reproduce/smri/cortical_vol.npy')\n\ntbss_ad = np.load('for_reproduce/dmri/tbss/tbss_ad.npy')\ntbss_fa = np.load('for_reproduce/dmri/tbss/tbss_fa.npy')\ntbss_icvf = np.load('for_reproduce/dmri/tbss/tbss_icvf.npy')\ntbss_isovf = np.load('for_reproduce/dmri/tbss/tbss_isovf.npy')\ntbss_md = np.load('for_reproduce/dmri/tbss/tbss_md.npy')\ntbss_od = np.load('for_reproduce/dmri/tbss/tbss_od.npy')\ntbss_rd = np.load('for_reproduce/dmri/tbss/tbss_rd.npy')\n\ntractography_ad = np.load('for_reproduce/dmri/tracto/tracto_ad.npy')\ntractography_fa = np.load('for_reproduce/dmri/tracto/tracto_fa.npy')\ntractography_icvf = np.load('for_reproduce/dmri/tracto/tracto_icvf.npy')\ntractography_isovf = np.load('for_reproduce/dmri/tracto/tracto_isovf.npy')\ntractography_md = np.load('for_reproduce/dmri/tracto/tracto_md.npy')\ntractography_od = np.load('for_repro

In [20]:
cort_thick = pd.read_csv('./for_reproduce/smri/cort_thick_pca_thres8.csv', encoding = 'cp949')
cort_area = pd.read_csv('for_reproduce/smri/cort_area_pca_thres8.csv', encoding = 'cp949')
cort_vol = pd.read_csv('for_reproduce/smri/cort_vol_pca_thres8.csv', encoding = 'cp949')

tbss_ad = pd.read_csv('for_reproduce/dmri/tbss/tbss_ad_pca_thres8.csv', encoding = 'cp949')
tbss_fa = pd.read_csv('for_reproduce/dmri/tbss/tbss_fa_pca_thres8.csv', encoding = 'cp949')
tbss_icvf = pd.read_csv('for_reproduce/dmri/tbss/tbss_icvf_pca_thres8.csv', encoding = 'cp949')
tbss_isovf = pd.read_csv('for_reproduce/dmri/tbss/tbss_isovf_pca_thres8.csv', encoding = 'cp949')
tbss_md = pd.read_csv('for_reproduce/dmri/tbss/tbss_md_pca_thres8.csv', encoding = 'cp949')
tbss_od = pd.read_csv('for_reproduce/dmri/tbss/tbss_od_pca_thres8.csv', encoding = 'cp949')
tbss_rd = pd.read_csv('for_reproduce/dmri/tbss/tbss_rd_pca_thres8.csv', encoding = 'cp949')

tractography_ad = pd.read_csv('for_reproduce/dmri/tracto/tractography_ad_pca_thres8.csv', encoding = 'cp949')
tractography_fa = pd.read_csv('for_reproduce/dmri/tracto/tractography_fa_pca_thres8.csv', encoding = 'cp949')
tractography_icvf = pd.read_csv('for_reproduce/dmri/tracto/tractography_icvf_pca_thres8.csv', encoding = 'cp949')
tractography_isovf = pd.read_csv('for_reproduce/dmri/tracto/tractography_isovf_pca_thres8.csv', encoding = 'cp949')
tractography_md = pd.read_csv('for_reproduce/dmri/tracto/tractography_md_pca_thres8.csv', encoding = 'cp949')
tractography_od = pd.read_csv('for_reproduce/dmri/tracto/tractography_od_pca_thres8.csv', encoding = 'cp949')
tractography_rd = pd.read_csv('for_reproduce/dmri/tracto/tractography_rd_pca_thres8.csv', encoding = 'cp949')
tractography_fss = pd.read_csv('for_reproduce/dmri/tracto/tractography_fss_pca_thres8.csv', encoding = 'cp949')
tractography_fssl = pd.read_csv('for_reproduce/dmri/tracto/tractography_fssl_pca_thres8.csv', encoding = 'cp949')

f_gamb = pd.read_csv('for_reproduce/fmri/f_gamb_pca_thres8.csv', encoding = 'cp949')
f_lang = pd.read_csv('for_reproduce/fmri/f_lang_pca_thres8.csv', encoding = 'cp949')
f_motor = pd.read_csv('for_reproduce/fmri/f_motor_pca_thres8.csv', encoding = 'cp949')
f_rs = pd.read_csv('for_reproduce/fmri/f_rs_pca_thres8.csv', encoding = 'cp949')
f_social = pd.read_csv('for_reproduce/fmri/f_social_pca_thres8.csv', encoding = 'cp949')
f_wm = pd.read_csv('for_reproduce/fmri/f_wm_pca_thres8.csv', encoding = 'cp949')

In [21]:
pd.DataFrame(cort_thick).T.head()

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
Unnamed: 0,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,pca11,...,pca125,pca126,pca127,pca128,pca129,pca130,pca131,pca132,pca133,pca134
설명가능한 분산 비율(고윳값),96.849582,13.88851,12.197636,9.518951,5.852399,5.54741,4.761835,4.111414,3.793825,3.653597,...,0.726122,0.719266,0.706115,0.700596,0.695358,0.690093,0.687899,0.67268,0.670891,0.656478
기여율,0.241802,0.034675,0.030454,0.023766,0.014612,0.01385,0.011889,0.010265,0.009472,0.009122,...,0.001813,0.001796,0.001763,0.001749,0.001736,0.001723,0.001717,0.001679,0.001675,0.001639
누적기여율,0.241802,0.276478,0.306931,0.330697,0.345309,0.359159,0.371047,0.381312,0.390784,0.399906,...,0.786199,0.787994,0.789757,0.791507,0.793243,0.794966,0.796683,0.798362,0.800037,0.801676


In [14]:
gpu = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [16]:
def make_meta(df, pheno_with_age, subject_list, seed):
    random.seed(seed)
    df = pd.DataFrame(df).T
    df.index = subject_list
    
    random.shuffle(subject_list)
    
    df = df.reindex(subject_list)
    pheno_with_age = pheno_with_age.reindex(subject_list)
    
    meta_x_data = df.to_numpy()
    meta_y_data = pheno_with_age.to_numpy()
    
    return meta_x_data, meta_y_data

In [13]:
def dataset_preprocessing(meta_x_data, meta_y_data, k_num, seed):
    
    x_train, x_test, y_train, y_test = train_test_split(meta_x_data, meta_y_data, test_size=0.2, random_state=seed)
    
    # Train meta set을 Train / Validation set split 
    train_meta_x = x_train[:int(x_train.shape[0] * 0.8)]
    val_meta_x = x_train[int(x_train.shape[0] * 0.8):]
    train_meta_y = y_train[:int(y_train.shape[0] * 0.8), :-1]
    val_meta_y = y_train[int(y_train.shape[0] * 0.8):, :-1]
    
    # Train set과 Validation set의 pheno_with_age를 StandardScaling
    train_scaler = StandardScaler()
    train_scaler.fit(train_meta_y)
    train_meta_y = train_scaler.transform(train_meta_y)
    val_meta_y = train_scaler.transform(val_meta_y)
    
    # Test meta set을 K-train과 rest_train_meta로 분리 
    k_train_meta = x_test[:k_num] # k * smri(400) 
    rest_train_meta = x_test[k_num:] # rest * smri(400)
    
    
    test_scaler = StandardScaler()
    test_scaler.fit(y_test[:k_num])
    k_shot_meta_y = test_scaler.transform(y_test[:k_num])
    test_meta_y = test_scaler.transform(y_test[k_num:])        

    # _val_ : Age를 제외한 Phenotype
    # _test_ : Age에 대한 Series 
    k_val_meta = k_shot_meta_y[:, :-1] # k * 58
    k_test_meta = k_shot_meta_y[:, -1] # k * age(1)
    rest_val_meta = test_meta_y[:, :-1] # rest * 58
    rest_test_meta = test_meta_y[:, -1] # rest * age(1)
    
    return train_meta_x, val_meta_x, train_meta_y, val_meta_y, k_train_meta, rest_train_meta, k_val_meta, rest_val_meta, k_test_meta, rest_test_meta

In [17]:
def dataset_finetune_preprocessing(meta_x_data, meta_y_data, k_num, seed):
    
    x_train, x_test, y_train, y_test = train_test_split(meta_x_data, meta_y_data, test_size=0.2, random_state=seed)
    
    # Train meta set을 Train / Validation set split 
    train_meta_x = x_train[:int(x_train.shape[0] * 0.8)]
    val_meta_x = x_train[int(x_train.shape[0] * 0.8):]
    train_meta_y = y_train[:int(y_train.shape[0] * 0.8), :-1]
    val_meta_y = y_train[int(y_train.shape[0] * 0.8):, :-1]
    
    # Train set과 Validation set의 pheno_with_age를 StandardScaling
    train_scaler = StandardScaler()
    train_scaler.fit(train_meta_y)
    train_meta_y = train_scaler.transform(train_meta_y)
    val_meta_y = train_scaler.transform(val_meta_y)
    
    # Test meta set을 K-train과 rest_train_meta로 분리 
    k_train_meta = x_test[:k_num] # k * smri(400) 
    rest_train_meta = x_test[k_num:] # rest * smri(400)
    

    
    test_scaler = StandardScaler()
    test_scaler.fit(y_test[:k_num])
    k_shot_meta_y = test_scaler.transform(y_test[:k_num])
    test_meta_y = test_scaler.transform(y_test[k_num:])        

    # _val_ : Age를 제외한 Phenotype
    # _test_ : Age에 대한 Series 
    k_val_meta = k_shot_meta_y[:, :-1] # k * 58
    k_test_meta = k_shot_meta_y[:, -1] # k * age(1)
    rest_val_meta = test_meta_y[:, :-1] # rest * 58
    rest_test_meta = test_meta_y[:, -1] # rest * age(1)
    
    # k-shot을 finetuning 위해 train/validation split
    ft_train_x, ft_val_x, ft_train_y, ft_val_y = train_test_split(k_train_meta, k_shot_meta_y, test_size=0.2, random_state=seed)
    ft_age_x = ft_train_y[:, -1]
    ft_age_y = ft_val_y[:, -1]
    ft_train_y = ft_train_y[:, :-1]
    ft_val_y = ft_val_y[:, :-1]
            
    return train_meta_x, val_meta_x, train_meta_y, val_meta_y, k_train_meta, rest_train_meta, k_val_meta, rest_val_meta, k_test_meta, rest_test_meta, ft_train_x, ft_val_x, ft_train_y, ft_val_y, ft_age_x, ft_age_y

In [18]:
def advanced_finetune(anat_name ,anat, k_num, iteration = 10):
    
    corres = []
    
    # 반복 실험 
    for seed in range(1, iteration+1):
        
        # Random Seed 설정
        print('iter ', seed)
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)        
        # CUDA를 사용하는 경우 => 이거 반복적으로 동일한 결과가 나오는 지 확인 
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

        # 모델 가중치 파일 이름 설정
        model_pth = anat_name + '_'+str(seed) +'_k'+ str(k_num) +'_dnn4l.pth'
        
        # Anat과 pheno_with_age의 index를 랜덤하게 Shuffling -> Train meta set과 Test meta set으로 분리 
        meta_x_data, meta_y_data = make_meta(anat, pheno_with_age, subject_list, seed)        
        train_meta_x, val_meta_x, train_meta_y, val_meta_y, k_train_meta, rest_train_meta, k_val_meta, rest_val_meta, k_test_meta, rest_test_meta, ft_train_x, ft_val_x, ft_train_y, ft_val_y, ft_age_x, ft_age_y = dataset_finetune_preprocessing(meta_x_data, meta_y_data, k_num, seed)
        
        print(val_meta_x.shape, val_meta_y.shape)a
        # train
        train_meta_x = torch.Tensor(train_meta_x).to(device)
        train_meta_y = torch.Tensor(train_meta_y).to(device)
        
        # validation
        val_meta_x = torch.Tensor(val_meta_x).to(device)
        val_meta_y = torch.Tensor(val_meta_y).to(device)
        
        # k-shot
        k_train_meta = torch.Tensor(k_train_meta).to(device)
        k_val_meta = torch.Tensor(k_val_meta).to(device)
        
        # Rest (Test meta set에서 K-shot을 제외한 나머지 samples)
        rest_train_meta = torch.Tensor(rest_train_meta).to(device)
        rest_val_meta = torch.Tensor(rest_val_meta).to(device)
        
        # fine tune train
        ft_train_x = torch.Tensor(ft_train_x).to(device)
        ft_train_y = torch.Tensor(ft_train_y).to(device)
        
        # fine tune validation
        ft_val_x = torch.Tensor(ft_val_x).to(device)
        ft_val_y = torch.Tensor(ft_val_y).to(device)        
        
        train_dataset = torch.utils.data.TensorDataset(train_meta_x, train_meta_y)
        val_dataset = torch.utils.data.TensorDataset(val_meta_x, val_meta_y)
        train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
        val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
        k_train_set = torch.utils.data.TensorDataset(k_train_meta, k_val_meta)
        rest_train_set = torch.utils.data.TensorDataset(rest_train_meta, rest_val_meta)
        k_train_dataloader = DataLoader(k_train_set, batch_size = k_train_meta.shape[0], shuffle=True, drop_last=True)
        rest_train_dataloader = DataLoader(rest_train_set, batch_size=16, shuffle=True)
        
        ft_train_dataset = torch.utils.data.TensorDataset(ft_train_x, ft_train_y)
        ft_val_dataset = torch.utils.data.TensorDataset(ft_val_x, ft_val_y)
        ft_train_dataloader = DataLoader(ft_train_dataset, batch_size= ft_train_x.shape[0], shuffle=True, drop_last=True)
        ft_val_dataloader = DataLoader(ft_val_dataset, batch_size=16, shuffle=False)

        # 모델 정의 및 Loss function & Optimizer
        model = dnn_4l(train_meta_x.shape[1], 256, 128, 64, 0.2, train_meta_y.shape[1]).to(device)
        model.to(device)
        loss_function = nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        
        # Meta train : Training (100 Epoch)
        for epoch in range(100):  
            model.train()
            for inputs, targets in train_dataloader:
                optimizer.zero_grad()
                outputs = model(inputs) # (batch_size, 58)
                loss = loss_function(outputs, targets)
                loss.backward()
                optimizer.step()
                    
            # Epoch 10 단위마다 출력 
            # if epoch % 10 == 9:
                # print(f"Epoch {epoch+1} - Train Loss: {loss:.4f}")
                
            if epoch == 99:
                torch.save(model, model_pth)  
        
        
        # validation
        model.eval()
        # Validation prediction
        val_outputs = []
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                outputs = model(inputs)
                val_outputs.append(outputs)
        val_outputs = torch.cat(val_outputs).cpu().detach().numpy()
        output_mat = val_outputs.T
        validation_mat = val_meta_y.detach().cpu().numpy().T
        
        # 긱 Phenotype마다, Correlation Coefficient를 저장하기 위한 리스트 
        val_pheno_corrs = []
        for i in range(len(output_mat)):
            val_pheno_corrs.append(np.corrcoef(output_mat[i], validation_mat[i])[0][1]) 
            
        print("=====Validation=====")
        print('correlation :',sum(val_pheno_corrs)/len(val_pheno_corrs))
        print('max correlation index :', val_pheno_corrs.index(max(val_pheno_corrs)))
        
        
        
        # k shot Prediction
        model.eval()
        k_train_outputs = []        
        with torch.no_grad():
            for inputs, targets in k_train_dataloader:
                K_outputs = model(inputs)
                k_train_outputs.append(K_outputs)
        k_train_outputs = torch.cat(k_train_outputs).cpu().detach().numpy() # k_train_outputs에는 (k_num, 58)의 matrix가 형성? 
        k_shot_age_corrs = []
        k_train_outputs = k_train_outputs.T
        for i in range(len(k_train_outputs)):
            k_shot_age_corrs.append(np.corrcoef(k_train_outputs[i], k_test_meta)[0][1]) 
        
        print("=====K-shot=====")
        print('k-shot correlation :',sum(k_shot_age_corrs)/len(k_shot_age_corrs))
        print('k-shot max correlation index :', k_shot_age_corrs.index(max(k_shot_age_corrs)))
            
            
        # k-shot max correlation index 
        max_corr_pheno_index = k_shot_age_corrs.index(max(k_shot_age_corrs))
        print('max corr pheno index', max_corr_pheno_index)
        
        
        # k shot Fine Tuning train
        
        # last 2 layers update
        optimizer = torch.optim.SGD([{'params': model.fc3.parameters()}, {'params': model.fc4.parameters()}],
                                    lr = 0.01, momentum = 0.9)
        for epoch in range(100):  
            model.train()
            for inputs, targets in ft_train_dataloader:
                optimizer.zero_grad()
                outputs = model(inputs) # (batch_size, 58)
                loss = loss_function(outputs, targets)
                loss.backward()
                optimizer.step()
                    
            # Epoch 10 단위마다 출력 
            # if epoch % 10 == 9:
                # print(f"Epoch {epoch+1} - Train Loss: {loss:.4f}")
                
            if epoch == 99:
                torch.save(model, model_pth)  
                
        # k shot Fine Tuning validation
        model.eval()
        
        val_outputs = []
        with torch.no_grad():
            for inputs, targets in ft_val_dataloader:
                outputs = model(inputs)
                val_outputs.append(outputs)
        val_outputs = torch.cat(val_outputs).cpu().detach().numpy()
        val_outputs = val_outputs[:, max_corr_pheno_index].reshape(-1, 1)
        age_corr = np.corrcoef(val_outputs[:, 0], ft_age_y)[0][1]
        print("=====K-shot fine tuning validation=====")        
        print('age corr', age_corr)


        # rest Prediction
   
        rest_outputs = [] 
        with torch.no_grad():
            for inputs, targets in rest_train_dataloader:
                outputs = model(inputs)
                rest_outputs.append(outputs)
        rest_outputs = torch.cat(rest_outputs).cpu().detach().numpy()
        # k shot에서 max correlation index에 해당하는 output만 가져옴
        rest_outputs = rest_outputs[:, max_corr_pheno_index].reshape(-1, 1)
        age_corr = np.corrcoef(rest_outputs[:, 0], rest_test_meta)[0][1]
        print("=====Test=====")
        print('age corr :', age_corr)
        corres.append(age_corr)            
        
    if iteration > 1 : 
        print(np.mean(corres))
        print(np.std(corres)) 
        
    print("===========================================================")

In [19]:
advanced_finetune('cort_thick' ,cort_thick, 10, 10)
advanced_finetune('cort_thick' ,cort_thick, 30, 10)
advanced_finetune('cort_thick' ,cort_thick, 50, 10)

iter  1
(121, 400) (121, 58)
=====Validation=====
correlation : 0.02245625407035938
max correlation index : 5
=====K-shot=====
k-shot correlation : 0.0048388815231033885
k-shot max correlation index : 18
max corr pheno index 18
=====K-shot fine tuning validation=====
age corr 0.9999999999999998
=====Test=====
age corr : -0.08402527627593566
iter  2
(121, 400) (121, 58)
=====Validation=====
correlation : -0.005279855581782694
max correlation index : 19
=====K-shot=====
k-shot correlation : 0.06576602346590453
k-shot max correlation index : 17
max corr pheno index 17
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.13302298517467856
iter  3
(121, 400) (121, 58)
=====Validation=====
correlation : 0.004806775079642398
max correlation index : 11
=====K-shot=====
k-shot correlation : -0.020689854856117858
k-shot max correlation index : 21
max corr pheno index 21
=====K-shot fine tuning validation=====
age corr 0.9999999999999999
=====Test=====
age corr : 0.050

In [20]:
advanced_finetune('cort_area' ,cort_area, 10, 10)
advanced_finetune('cort_area' ,cort_area, 30, 10)
advanced_finetune('cort_area' ,cort_area, 50, 10)

iter  1
(121, 400) (121, 58)
=====Validation=====
correlation : 0.0007054979782590979
max correlation index : 10
=====K-shot=====
k-shot correlation : -0.008525929137885807
k-shot max correlation index : 48
max corr pheno index 48
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.1550763524595797
iter  2
(121, 400) (121, 58)
=====Validation=====
correlation : -0.003014087790840545
max correlation index : 30
=====K-shot=====
k-shot correlation : 0.12619999622854922
k-shot max correlation index : 3
max corr pheno index 3
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.07749047933599587
iter  3
(121, 400) (121, 58)
=====Validation=====
correlation : -0.014979640677927078
max correlation index : 7
=====K-shot=====
k-shot correlation : 0.03766568032846302
k-shot max correlation index : 44
max corr pheno index 44
=====K-shot fine tuning validation=====
age corr nan
=====Test=====
age corr : -0.08961143324852372
iter  4
(121, 

In [21]:
advanced_finetune('cort_vol' ,cort_vol, 10, 10)
advanced_finetune('cort_vol' ,cort_vol, 30, 10)
advanced_finetune('cort_vol' ,cort_vol, 50, 10)

iter  1
(121, 400) (121, 58)
=====Validation=====
correlation : -0.005783656156829319
max correlation index : 5
=====K-shot=====
k-shot correlation : -0.04090430815552012
k-shot max correlation index : 6
max corr pheno index 6
=====K-shot fine tuning validation=====
age corr 0.9999999999999999
=====Test=====
age corr : 0.12880788132396992
iter  2
(121, 400) (121, 58)
=====Validation=====
correlation : -0.013885318850034006
max correlation index : 19
=====K-shot=====
k-shot correlation : 0.11220696500546426
k-shot max correlation index : 2
max corr pheno index 2
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.16566570150197166
iter  3
(121, 400) (121, 58)
=====Validation=====
correlation : 0.006057357445208264
max correlation index : 23
=====K-shot=====
k-shot correlation : -0.07948382574651494
k-shot max correlation index : 57
max corr pheno index 57
=====K-shot fine tuning validation=====
age corr 0.9999999999999999
=====Test=====
age corr : -0.002717

In [22]:
advanced_finetune('cort_vol' ,cort_vol, 10, 10)
advanced_finetune('cort_vol' ,cort_vol, 30, 10)
advanced_finetune('cort_vol' ,cort_vol, 50, 10)

iter  1
(121, 133053) (121, 58)
=====Validation=====
correlation : 0.027479171547559814
max correlation index : 52
=====K-shot=====
k-shot correlation : -0.09816262533733168
k-shot max correlation index : 16
max corr pheno index 16
=====K-shot fine tuning validation=====
age corr -0.9999999999999999
=====Test=====
age corr : 0.18935256943259365
iter  2
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.04894338689984077
max correlation index : 45
=====K-shot=====
k-shot correlation : 0.07636907810846516
k-shot max correlation index : 45
max corr pheno index 45
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.09193286022970583
iter  3
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.019767064624274838
max correlation index : 32
=====K-shot=====
k-shot correlation : 0.09545207900509675
k-shot max correlation index : 8
max corr pheno index 8
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.0619358776

In [24]:
advanced_finetune('tbss_fa' ,tbss_fa, 10, 10)
advanced_finetune('tbss_fa' ,tbss_fa, 30, 10)
advanced_finetune('tbss_fa' ,tbss_fa, 50, 10)

iter  1
(121, 133053) (121, 58)
=====Validation=====
correlation : 0.016273668939824758
max correlation index : 25
=====K-shot=====
k-shot correlation : 0.11212700183898354
k-shot max correlation index : 0
max corr pheno index 0
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.03570749833573005
iter  2
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.006026179754780866
max correlation index : 10
=====K-shot=====
k-shot correlation : -0.02536283158268145
k-shot max correlation index : 19
max corr pheno index 19
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.08856964638929629
iter  3
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.005232538487425165
max correlation index : 52
=====K-shot=====
k-shot correlation : 0.04799859085527619
k-shot max correlation index : 11
max corr pheno index 11
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.07427301013289211
iter  

In [25]:
advanced_finetune('tbss_icvf' ,tbss_icvf, 10, 10)
advanced_finetune('tbss_icvf' ,tbss_icvf, 30, 10)
advanced_finetune('tbss_icvf' ,tbss_icvf, 50, 10)

iter  1
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.061933564395196566
max correlation index : 2
=====K-shot=====
k-shot correlation : 0.02941868310352936
k-shot max correlation index : 17
max corr pheno index 17
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.039974422054308754
iter  2
(121, 133053) (121, 58)
=====Validation=====
correlation : 0.01384188036738978
max correlation index : 57
=====K-shot=====
k-shot correlation : 0.18716623090375015
k-shot max correlation index : 35
max corr pheno index 35
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.09119115461658826
iter  3
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.012023004890655616
max correlation index : 26
=====K-shot=====
k-shot correlation : 0.023408809599746665
k-shot max correlation index : 28
max corr pheno index 28
=====K-shot fine tuning validation=====
age corr 0.9999999999999998
=====Test=====
age corr : 0.084649591

In [26]:
advanced_finetune('tbss_isovf' ,tbss_isovf, 10, 10)
advanced_finetune('tbss_isovf' ,tbss_isovf, 30, 10)
advanced_finetune('tbss_isovf' ,tbss_isovf, 50, 10)

iter  1
(121, 133053) (121, 58)
=====Validation=====
correlation : 0.008439894158475366
max correlation index : 17
=====K-shot=====
k-shot correlation : -0.11246097187815296
k-shot max correlation index : 32
max corr pheno index 32
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.18801514964040875
iter  2
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.015464355105568877
max correlation index : 17
=====K-shot=====
k-shot correlation : 0.03572400150168154
k-shot max correlation index : 28
max corr pheno index 28
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.05442926153705955
iter  3
(121, 133053) (121, 58)
=====Validation=====
correlation : 0.0038207157780287587
max correlation index : 13
=====K-shot=====
k-shot correlation : 0.01648108393912211
k-shot max correlation index : 53
max corr pheno index 53
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.019490617842933013
it

In [27]:
advanced_finetune('tbss_md' ,tbss_md, 10, 10)
advanced_finetune('tbss_md' ,tbss_md, 30, 10)
advanced_finetune('tbss_md' ,tbss_md, 50, 10)

iter  1
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.013461947507412439
max correlation index : 10
=====K-shot=====
k-shot correlation : -0.08730011028049535
k-shot max correlation index : 21
max corr pheno index 21
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.06577578051692164
iter  2
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.016709931406035713
max correlation index : 27
=====K-shot=====
k-shot correlation : 0.11387701177507167
k-shot max correlation index : 4
max corr pheno index 4
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.07903067663166788
iter  3
(121, 133053) (121, 58)
=====Validation=====
correlation : 0.012604616460220859
max correlation index : 45
=====K-shot=====
k-shot correlation : -0.027943965221004993
k-shot max correlation index : 30
max corr pheno index 30
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.0905795040845926
iter

In [28]:
advanced_finetune('tbss_od' ,tbss_od, 10, 10)
advanced_finetune('tbss_od' ,tbss_od, 30, 10)
advanced_finetune('tbss_od' ,tbss_od, 50, 10)

iter  1
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.006420598885453066
max correlation index : 8
=====K-shot=====
k-shot correlation : 0.026074911180770828
k-shot max correlation index : 14
max corr pheno index 14
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.004821129189134691
iter  2
(121, 133053) (121, 58)
=====Validation=====
correlation : 0.049718219526642
max correlation index : 25
=====K-shot=====
k-shot correlation : 0.0526606601751668
k-shot max correlation index : 52
max corr pheno index 52
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.048432190748392556
iter  3
(121, 133053) (121, 58)
=====Validation=====
correlation : 0.007770643122333402
max correlation index : 9
=====K-shot=====
k-shot correlation : 0.03462176286580825
k-shot max correlation index : 23
max corr pheno index 23
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.017235246332666116
iter  4
(

In [29]:
advanced_finetune('tbss_rd' ,tbss_rd, 10, 10)
advanced_finetune('tbss_rd' ,tbss_rd, 30, 10)
advanced_finetune('tbss_rd' ,tbss_rd, 50, 10)

iter  1
(121, 133053) (121, 58)
=====Validation=====
correlation : 0.012301727778509023
max correlation index : 51
=====K-shot=====
k-shot correlation : 0.07341917874667216
k-shot max correlation index : 5
max corr pheno index 5
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.00474435981630008
iter  2
(121, 133053) (121, 58)
=====Validation=====
correlation : 0.0013922480570434834
max correlation index : 32
=====K-shot=====
k-shot correlation : -0.018094550323942547
k-shot max correlation index : 30
max corr pheno index 30
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.12129582378444044
iter  3
(121, 133053) (121, 58)
=====Validation=====
correlation : -0.02572385003276315
max correlation index : 32
=====K-shot=====
k-shot correlation : 0.19595911632982757
k-shot max correlation index : 8
max corr pheno index 8
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.024112355649179212
iter  4


In [30]:
advanced_finetune('tractography_ad' ,tractography_ad, 10, 10)
advanced_finetune('tractography_ad' ,tractography_ad, 30, 10)
advanced_finetune('tractography_ad' ,tractography_ad, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.020696554043011822
max correlation index : 32
=====K-shot=====
k-shot correlation : 0.047031740102976824
k-shot max correlation index : 49
max corr pheno index 49
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.031998745360037786
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.01200060494355236
max correlation index : 4
=====K-shot=====
k-shot correlation : -0.025289794098762777
k-shot max correlation index : 57
max corr pheno index 57
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.03743727022614822
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.012174931588712436
max correlation index : 10
=====K-shot=====
k-shot correlation : -0.12098041814218305
k-shot max correlation index : 18
max corr pheno index 18
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.0058400172843744045
iter  

In [31]:
advanced_finetune('tractography_fa' ,tractography_fa, 10, 10)
advanced_finetune('tractography_fa' ,tractography_fa, 30, 10)
advanced_finetune('tractography_fa' ,tractography_fa, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.008279409506792613
max correlation index : 47
=====K-shot=====
k-shot correlation : -0.06196648723030353
k-shot max correlation index : 17
max corr pheno index 17
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.13541074030500358
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.01587258626384148
max correlation index : 18
=====K-shot=====
k-shot correlation : 0.11316964370162379
k-shot max correlation index : 42
max corr pheno index 42
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.06473697182417117
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.026040326029400186
max correlation index : 51
=====K-shot=====
k-shot correlation : 0.024936220146370144
k-shot max correlation index : 3
max corr pheno index 3
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.0014500399022673398
iter  4
(12

In [32]:
advanced_finetune('tractography_icvf' ,tractography_icvf, 10, 10)
advanced_finetune('tractography_icvf' ,tractography_icvf, 30, 10)
advanced_finetune('tractography_icvf' ,tractography_icvf, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.012889513318164724
max correlation index : 54
=====K-shot=====
k-shot correlation : 0.06951052898291202
k-shot max correlation index : 15
max corr pheno index 15
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.03768924666297455
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.005184455927403416
max correlation index : 13
=====K-shot=====
k-shot correlation : 0.1629707651643043
k-shot max correlation index : 5
max corr pheno index 5
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.0026208211815115052
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.00505585437035962
max correlation index : 23
=====K-shot=====
k-shot correlation : 0.02342525822979684
k-shot max correlation index : 36
max corr pheno index 36
=====K-shot fine tuning validation=====
age corr -0.9999999999999999
=====Test=====
age corr : 0.08799357699769

In [33]:
advanced_finetune('tractography_isovf' ,tractography_isovf, 10, 10)
advanced_finetune('tractography_isovf' ,tractography_isovf, 30, 10)
advanced_finetune('tractography_isovf' ,tractography_isovf, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.016863853979933043
max correlation index : 19
=====K-shot=====
k-shot correlation : 0.19985555125103646
k-shot max correlation index : 29
max corr pheno index 29
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.03759435772770548
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.011222620514422814
max correlation index : 34
=====K-shot=====
k-shot correlation : -0.019567379092140968
k-shot max correlation index : 7
max corr pheno index 7
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.140497100840578
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.02116955607081695
max correlation index : 36
=====K-shot=====
k-shot correlation : -0.04170449906974931
k-shot max correlation index : 57
max corr pheno index 57
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.0005575521222101911
iter  4
(121

In [34]:
advanced_finetune('tractography_md' ,tractography_md, 10, 10)
advanced_finetune('tractography_md' ,tractography_md, 30, 10)
advanced_finetune('tractography_md' ,tractography_md, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.027251369000818374
max correlation index : 53
=====K-shot=====
k-shot correlation : 0.1111442586642178
k-shot max correlation index : 3
max corr pheno index 3
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.16403660928299346
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.023787533027050344
max correlation index : 25
=====K-shot=====
k-shot correlation : -0.09862489562712466
k-shot max correlation index : 47
max corr pheno index 47
=====K-shot fine tuning validation=====
age corr 0.9999999999999998
=====Test=====
age corr : -0.17203335530130925
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.04611020894111277
max correlation index : 25
=====K-shot=====
k-shot correlation : 0.026201645987148397
k-shot max correlation index : 2
max corr pheno index 2
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.1261316284470505

In [35]:
advanced_finetune('tractography_od' ,tractography_od, 10, 10)
advanced_finetune('tractography_od' ,tractography_od, 30, 10)
advanced_finetune('tractography_od' ,tractography_od, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.013789618480931576
max correlation index : 45
=====K-shot=====
k-shot correlation : -0.0888904309771046
k-shot max correlation index : 43
max corr pheno index 43
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.021373999434637133
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.0008916713760275474
max correlation index : 11
=====K-shot=====
k-shot correlation : -0.09414546992690846
k-shot max correlation index : 32
max corr pheno index 32
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.09032723509159413
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.022024584558970273
max correlation index : 52
=====K-shot=====
k-shot correlation : 0.17160308274436592
k-shot max correlation index : 7
max corr pheno index 7
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.07809168744904452
iter  4
(

In [36]:
advanced_finetune('tractography_rd' ,tractography_rd, 10, 10)
advanced_finetune('tractography_rd' ,tractography_rd, 30, 10)
advanced_finetune('tractography_rd' ,tractography_rd, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.005829534716647673
max correlation index : 17
=====K-shot=====
k-shot correlation : 0.01828027573626699
k-shot max correlation index : 30
max corr pheno index 30
=====K-shot fine tuning validation=====
age corr 0.9999999999999999
=====Test=====
age corr : 0.073390270505687
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.0012541149877022448
max correlation index : 9
=====K-shot=====
k-shot correlation : -0.16654295812720815
k-shot max correlation index : 21
max corr pheno index 21
=====K-shot fine tuning validation=====
age corr -0.9999999999999999
=====Test=====
age corr : -0.054730478570404834
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.01714087513811619
max correlation index : 55
=====K-shot=====
k-shot correlation : 0.017049228560329508
k-shot max correlation index : 17
max corr pheno index 17
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr :

In [37]:
advanced_finetune('tractography_fss' ,tractography_fss, 10, 10)
advanced_finetune('tractography_fss' ,tractography_fss, 30, 10)
advanced_finetune('tractography_fss' ,tractography_fss, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.0013538092802451095
max correlation index : 20
=====K-shot=====
k-shot correlation : 0.009038899250587165
k-shot max correlation index : 24
max corr pheno index 24
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.0861414323199506
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.007664550313177098
max correlation index : 1
=====K-shot=====
k-shot correlation : -0.07754873668316688
k-shot max correlation index : 23
max corr pheno index 23
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.09786183479059368
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.012404296733065121
max correlation index : 53
=====K-shot=====
k-shot correlation : -0.02983433145970655
k-shot max correlation index : 48
max corr pheno index 48
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.18909353649806646
iter  4

In [38]:
advanced_finetune('tractography_fssl' ,tractography_fssl, 10, 10)
advanced_finetune('tractography_fssl' ,tractography_fssl, 30, 10)
advanced_finetune('tractography_fssl' ,tractography_fssl, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.010816968599399249
max correlation index : 38
=====K-shot=====
k-shot correlation : -0.14441335233258906
k-shot max correlation index : 18
max corr pheno index 18
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.09434663837501188
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.0012604026984184677
max correlation index : 50
=====K-shot=====
k-shot correlation : -0.07312369018641883
k-shot max correlation index : 21
max corr pheno index 21
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.039395273897569034
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.0053780812366766816
max correlation index : 38
=====K-shot=====
k-shot correlation : 0.0021793248484836348
k-shot max correlation index : 53
max corr pheno index 53
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.11386490122088513
it

In [39]:
advanced_finetune('f_gamb' ,f_gamb, 10, 10)
advanced_finetune('f_gamb' ,f_gamb, 30, 10)
advanced_finetune('f_gamb' ,f_gamb, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.026105940682374068
max correlation index : 1
=====K-shot=====
k-shot correlation : 0.043199842652187126
k-shot max correlation index : 27
max corr pheno index 27
=====K-shot fine tuning validation=====
age corr 0.9999999999999998
=====Test=====
age corr : -0.08648659770140768
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.03087937626309582
max correlation index : 52
=====K-shot=====
k-shot correlation : 0.11615412387600789
k-shot max correlation index : 55
max corr pheno index 55
=====K-shot fine tuning validation=====
age corr 0.9999999999999999
=====Test=====
age corr : 0.0054303368563588745
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.0001599443207792167
max correlation index : 6
=====K-shot=====
k-shot correlation : 0.022112178788162802
k-shot max correlation index : 42
max corr pheno index 42
=====K-shot fine tuning validation=====
age corr 0.9999999999999999
=====Test

In [40]:
advanced_finetune('f_lang' ,f_lang, 10, 10)
advanced_finetune('f_lang' ,f_lang, 30, 10)
advanced_finetune('f_lang' ,f_lang, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.038373561544863814
max correlation index : 56
=====K-shot=====
k-shot correlation : -0.011900427204702474
k-shot max correlation index : 55
max corr pheno index 55
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.04572426017002393
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.0005366113748799699
max correlation index : 50
=====K-shot=====
k-shot correlation : 0.021143097377364008
k-shot max correlation index : 32
max corr pheno index 32
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.05993423377592321
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.019014133629404885
max correlation index : 48
=====K-shot=====
k-shot correlation : 0.11446800120110052
k-shot max correlation index : 16
max corr pheno index 16
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.07507116748370068
iter 

In [41]:
advanced_finetune('f_motor' ,f_motor, 10, 10)
advanced_finetune('f_motor' ,f_motor, 30, 10)
advanced_finetune('f_motor' ,f_motor, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.0035758766189906335
max correlation index : 13
=====K-shot=====
k-shot correlation : -0.03544625246359419
k-shot max correlation index : 15
max corr pheno index 15
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.022506682185146027
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.011162289671846397
max correlation index : 14
=====K-shot=====
k-shot correlation : -0.07632781632672861
k-shot max correlation index : 42
max corr pheno index 42
=====K-shot fine tuning validation=====
age corr nan
=====Test=====
age corr : 0.13245038296414127
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.025055922610795336
max correlation index : 3
=====K-shot=====
k-shot correlation : 0.03774141099615653
k-shot max correlation index : 14
max corr pheno index 14
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.06786442299493303
iter  4

In [42]:
advanced_finetune('f_rs' ,f_rs, 10, 10)
advanced_finetune('f_rs' ,f_rs, 30, 10)
advanced_finetune('f_rs' ,f_rs, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.002340501131237041
max correlation index : 21
=====K-shot=====
k-shot correlation : 0.039170832168456265
k-shot max correlation index : 30
max corr pheno index 30
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.0851149160400593
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.024183305969369597
max correlation index : 34
=====K-shot=====
k-shot correlation : -0.08621747885534992
k-shot max correlation index : 22
max corr pheno index 22
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : 0.017545593493743034
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.007494097225005586
max correlation index : 15
=====K-shot=====
k-shot correlation : 0.1518170034960269
k-shot max correlation index : 33
max corr pheno index 33
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : 0.019552555824627102
iter  4


In [43]:
advanced_finetune('f_social' ,f_social, 10, 10)
advanced_finetune('f_social' ,f_social, 30, 10)
advanced_finetune('f_social' ,f_social, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.019689399566121472
max correlation index : 14
=====K-shot=====
k-shot correlation : -0.08075175744676236
k-shot max correlation index : 39
max corr pheno index 39
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.006898280702570028
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.02262137725076012
max correlation index : 45
=====K-shot=====
k-shot correlation : -0.18288272334166777
k-shot max correlation index : 27
max corr pheno index 27
=====K-shot fine tuning validation=====
age corr -0.9999999999999998
=====Test=====
age corr : -0.09785126011535339
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.032099525280750746
max correlation index : 9
=====K-shot=====
k-shot correlation : 0.07425955354512673
k-shot max correlation index : 13
max corr pheno index 13
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.1479814

In [44]:
advanced_finetune('f_wm' ,f_wm, 10, 10)
advanced_finetune('f_wm' ,f_wm, 30, 10)
advanced_finetune('f_wm' ,f_wm, 50, 10)

iter  1
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.007053038139542901
max correlation index : 12
=====K-shot=====
k-shot correlation : -0.14083958999606788
k-shot max correlation index : 23
max corr pheno index 23
=====K-shot fine tuning validation=====
age corr 1.0
=====Test=====
age corr : -0.049576934898522045
iter  2
(121, 79800) (121, 58)
=====Validation=====
correlation : 0.0007542688856011419
max correlation index : 8
=====K-shot=====
k-shot correlation : -0.08304104020006588
k-shot max correlation index : 53
max corr pheno index 53
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.0057495546388314995
iter  3
(121, 79800) (121, 58)
=====Validation=====
correlation : -0.0211133386250453
max correlation index : 41
=====K-shot=====
k-shot correlation : 0.12598075963747618
k-shot max correlation index : 3
max corr pheno index 3
=====K-shot fine tuning validation=====
age corr -1.0
=====Test=====
age corr : -0.08533718080525599
iter 

In [132]:
exp('cort_thick' ,cort_thick, 30, 10)

iter  1
(602, 400) (151, 400) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.4701 - Val Loss: 1.4914
Epoch 2 - Train Loss: 2.1274 - Val Loss: 1.4948
Epoch 3 - Train Loss: 1.9488 - Val Loss: 1.3821
Epoch 4 - Train Loss: 2.1023 - Val Loss: 1.4153
Epoch 5 - Train Loss: 2.0450 - Val Loss: 1.3663
Epoch 6 - Train Loss: 2.1479 - Val Loss: 1.4184
Epoch 7 - Train Loss: 1.8656 - Val Loss: 1.4254
Epoch 8 - Train Loss: 1.9538 - Val Loss: 1.3423
Epoch 9 - Train Loss: 1.9225 - Val Loss: 1.3464
Epoch 10 - Train Loss: 1.8173 - Val Loss: 1.4064
Epoch 11 - Train Loss: 1.7210 - Val Loss: 1.3486
Epoch 12 - Train Loss: 1.6786 - Val Loss: 1.3118
Epoch 13 - Train Loss: 1.6412 - Val Loss: 1.3011
Epoch 14 - Train Loss: 1.4709 - Val Loss: 1.3052
Epoch 15 - Train Loss: 1.8872 - Val Loss: 1.2977
Epoch 16 - Train Loss: 1.6507 - Val Loss: 1.2775
Epoch 17 - Train Loss: 1.4294 - Val Loss: 1.2836
Epoch 18 - Train Loss: 1.4001 - Val Loss: 1.2821
Epoch 19 - Train Loss: 1.4959 - Val Loss: 1.2757
Epoch 20 - Train Loss: 1.532

In [133]:
exp('cort_area' ,cort_area, 30, 10)

iter  1
(602, 400) (151, 400) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1759 - Val Loss: 1.3170
Epoch 2 - Train Loss: 2.0572 - Val Loss: 1.3252
Epoch 3 - Train Loss: 1.9295 - Val Loss: 1.3133
Epoch 4 - Train Loss: 2.0966 - Val Loss: 1.2916
Epoch 5 - Train Loss: 1.8026 - Val Loss: 1.2346
Epoch 6 - Train Loss: 2.0818 - Val Loss: 1.2033
Epoch 7 - Train Loss: 1.9607 - Val Loss: 1.2065
Epoch 8 - Train Loss: 1.7966 - Val Loss: 1.2027
Epoch 9 - Train Loss: 1.7363 - Val Loss: 1.1706
Epoch 10 - Train Loss: 1.6758 - Val Loss: 1.1532
Epoch 11 - Train Loss: 1.8403 - Val Loss: 1.1518
Epoch 12 - Train Loss: 1.5458 - Val Loss: 1.1382
Epoch 13 - Train Loss: 1.6765 - Val Loss: 1.1410
Epoch 14 - Train Loss: 1.5560 - Val Loss: 1.1320
Epoch 15 - Train Loss: 1.5893 - Val Loss: 1.1217
Epoch 16 - Train Loss: 1.5234 - Val Loss: 1.1197
Epoch 17 - Train Loss: 1.5988 - Val Loss: 1.1032
Epoch 18 - Train Loss: 1.5447 - Val Loss: 1.0958
Epoch 19 - Train Loss: 1.4645 - Val Loss: 1.0972
Epoch 20 - Train Loss: 1.509

In [134]:
exp('cort_vol' ,cort_vol, 30, 10)

iter  1
(602, 400) (151, 400) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1964 - Val Loss: 1.2782
Epoch 2 - Train Loss: 2.3177 - Val Loss: 1.2664
Epoch 3 - Train Loss: 1.7951 - Val Loss: 1.2673
Epoch 4 - Train Loss: 1.8151 - Val Loss: 1.2316
Epoch 5 - Train Loss: 2.0600 - Val Loss: 1.2172
Epoch 6 - Train Loss: 1.9887 - Val Loss: 1.1825
Epoch 7 - Train Loss: 2.1658 - Val Loss: 1.1823
Epoch 8 - Train Loss: 1.8768 - Val Loss: 1.1749
Epoch 9 - Train Loss: 2.0083 - Val Loss: 1.1291
Epoch 10 - Train Loss: 1.6715 - Val Loss: 1.1368
Epoch 11 - Train Loss: 1.7605 - Val Loss: 1.1368
Epoch 12 - Train Loss: 1.7779 - Val Loss: 1.1322
Epoch 13 - Train Loss: 1.6168 - Val Loss: 1.1127
Epoch 14 - Train Loss: 1.5581 - Val Loss: 1.0986
Epoch 15 - Train Loss: 1.4943 - Val Loss: 1.1098
Epoch 16 - Train Loss: 1.4068 - Val Loss: 1.0941
Epoch 17 - Train Loss: 1.5755 - Val Loss: 1.1022
Epoch 18 - Train Loss: 1.3669 - Val Loss: 1.0793
Epoch 19 - Train Loss: 1.2878 - Val Loss: 1.0754
Epoch 20 - Train Loss: 1.644

In [16]:
exp2('tbss_ad' ,tbss_ad, 30, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1344 - Val Loss: 1.2878
Epoch 2 - Train Loss: 2.2333 - Val Loss: 1.2730
Epoch 3 - Train Loss: 1.9943 - Val Loss: 4.6047
Epoch 4 - Train Loss: 1.9517 - Val Loss: 24.2011
Epoch 5 - Train Loss: 1.7983 - Val Loss: 2.1688
Epoch 6 - Train Loss: 1.6584 - Val Loss: 7.0811
Epoch 7 - Train Loss: 1.7530 - Val Loss: 1.5737
Epoch 8 - Train Loss: 1.6392 - Val Loss: 2.0019
Epoch 9 - Train Loss: 1.7190 - Val Loss: 1.6574
Epoch 10 - Train Loss: 1.3910 - Val Loss: 1.7688
Epoch 11 - Train Loss: 1.5643 - Val Loss: 1.3809
Epoch 12 - Train Loss: 1.4628 - Val Loss: 1.2241
Epoch 13 - Train Loss: 1.4883 - Val Loss: 1.1969
Epoch 14 - Train Loss: 1.4932 - Val Loss: 1.2524
Epoch 15 - Train Loss: 1.3901 - Val Loss: 1.1090
Epoch 16 - Train Loss: 1.3760 - Val Loss: 1.1397
Epoch 17 - Train Loss: 1.3531 - Val Loss: 1.1210
Epoch 18 - Train Loss: 1.2616 - Val Loss: 1.1019
Epoch 19 - Train Loss: 1.4068 - Val Loss: 1.0884
Epoch 20 - Train Loss

In [17]:
exp2('tbss_fa' ,tbss_fa , 30, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2792 - Val Loss: 11.3242
Epoch 2 - Train Loss: 2.0823 - Val Loss: 21.8777
Epoch 3 - Train Loss: 1.8850 - Val Loss: 66975.7988
Epoch 4 - Train Loss: 2.0786 - Val Loss: 2.0223
Epoch 5 - Train Loss: 1.8547 - Val Loss: 1.9599
Epoch 6 - Train Loss: 1.7298 - Val Loss: 1.3718
Epoch 7 - Train Loss: 1.6508 - Val Loss: 1.4041
Epoch 8 - Train Loss: 1.6246 - Val Loss: 1.8995
Epoch 9 - Train Loss: 1.6058 - Val Loss: 1.1486
Epoch 10 - Train Loss: 1.5330 - Val Loss: 1.1238
Epoch 11 - Train Loss: 1.3992 - Val Loss: 1.1337
Epoch 12 - Train Loss: 1.4996 - Val Loss: 1.0666
Epoch 13 - Train Loss: 1.3193 - Val Loss: 1.0543
Epoch 14 - Train Loss: 1.3958 - Val Loss: 1.0437
Epoch 15 - Train Loss: 1.3663 - Val Loss: 1.0429
Epoch 16 - Train Loss: 1.1829 - Val Loss: 1.0328
Epoch 17 - Train Loss: 1.2394 - Val Loss: 1.0042
Epoch 18 - Train Loss: 1.4014 - Val Loss: 1.0072
Epoch 19 - Train Loss: 1.3801 - Val Loss: 1.0071
Epoch 20 - Train

In [18]:
exp2('tbss_icvf' ,tbss_icvf , 30, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2932 - Val Loss: 3347.9436
Epoch 2 - Train Loss: 2.0053 - Val Loss: 1742.4630
Epoch 3 - Train Loss: 2.1781 - Val Loss: 5.1159
Epoch 4 - Train Loss: 1.8195 - Val Loss: 2.2330
Epoch 5 - Train Loss: 1.6908 - Val Loss: 3.2825
Epoch 6 - Train Loss: 1.7137 - Val Loss: 1.4724
Epoch 7 - Train Loss: 1.5414 - Val Loss: 1.8480
Epoch 8 - Train Loss: 1.4774 - Val Loss: 1.3237
Epoch 9 - Train Loss: 1.5490 - Val Loss: 1.2827
Epoch 10 - Train Loss: 1.4953 - Val Loss: 1.5513
Epoch 11 - Train Loss: 1.4570 - Val Loss: 4.3528
Epoch 12 - Train Loss: 1.3792 - Val Loss: 1.2784
Epoch 13 - Train Loss: 1.4022 - Val Loss: 1.1164
Epoch 14 - Train Loss: 1.2323 - Val Loss: 1.1107
Epoch 15 - Train Loss: 1.2982 - Val Loss: 1.1056
Epoch 16 - Train Loss: 1.4711 - Val Loss: 1.3038
Epoch 17 - Train Loss: 1.1992 - Val Loss: 1.0969
Epoch 18 - Train Loss: 1.4970 - Val Loss: 1.1005
Epoch 19 - Train Loss: 1.3294 - Val Loss: 1.0654
Epoch 20 - Train

In [19]:
exp2('tbss_isovf' ,tbss_isovf, 30, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1574 - Val Loss: 1.9321
Epoch 2 - Train Loss: 2.1336 - Val Loss: 1.4732
Epoch 3 - Train Loss: 1.6967 - Val Loss: 1.4045
Epoch 4 - Train Loss: 1.6423 - Val Loss: 1.3507
Epoch 5 - Train Loss: 1.4946 - Val Loss: 1.2858
Epoch 6 - Train Loss: 1.6677 - Val Loss: 1.2218
Epoch 7 - Train Loss: 1.5693 - Val Loss: 1.2326
Epoch 8 - Train Loss: 1.5027 - Val Loss: 1.1891
Epoch 9 - Train Loss: 1.2863 - Val Loss: 1.1821
Epoch 10 - Train Loss: 1.4302 - Val Loss: 1.1512
Epoch 11 - Train Loss: 1.1008 - Val Loss: 1.1519
Epoch 12 - Train Loss: 1.2454 - Val Loss: 1.1345
Epoch 13 - Train Loss: 1.1966 - Val Loss: 1.1366
Epoch 14 - Train Loss: 1.3288 - Val Loss: 1.1287
Epoch 15 - Train Loss: 1.1620 - Val Loss: 1.1120
Epoch 16 - Train Loss: 1.1708 - Val Loss: 1.1170
Epoch 17 - Train Loss: 1.1949 - Val Loss: 1.1067
Epoch 18 - Train Loss: 1.0359 - Val Loss: 1.1116
Epoch 19 - Train Loss: 1.4063 - Val Loss: 1.1050
Epoch 20 - Train Loss:

In [20]:
exp2('tbss_md' ,tbss_md , 30, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.3610 - Val Loss: 1.3621
Epoch 2 - Train Loss: 2.2095 - Val Loss: 1.5495
Epoch 3 - Train Loss: 2.1294 - Val Loss: 33.6456
Epoch 4 - Train Loss: 1.7472 - Val Loss: 9.0561
Epoch 5 - Train Loss: 1.9920 - Val Loss: 73.1708
Epoch 6 - Train Loss: 1.7234 - Val Loss: 210.7968
Epoch 7 - Train Loss: 1.6346 - Val Loss: 2.5114
Epoch 8 - Train Loss: 1.5010 - Val Loss: 29.0790
Epoch 9 - Train Loss: 1.5549 - Val Loss: 2.3831
Epoch 10 - Train Loss: 1.4528 - Val Loss: 1.5969
Epoch 11 - Train Loss: 1.4213 - Val Loss: 1.5805
Epoch 12 - Train Loss: 1.3982 - Val Loss: 1.2079
Epoch 13 - Train Loss: 1.4092 - Val Loss: 1.1912
Epoch 14 - Train Loss: 1.3564 - Val Loss: 1.2417
Epoch 15 - Train Loss: 1.4257 - Val Loss: 2.2554
Epoch 16 - Train Loss: 1.3900 - Val Loss: 1.2432
Epoch 17 - Train Loss: 1.2846 - Val Loss: 1.1572
Epoch 18 - Train Loss: 1.5036 - Val Loss: 1.1512
Epoch 19 - Train Loss: 1.3705 - Val Loss: 1.1546
Epoch 20 - Train 

In [21]:
exp2('tbss_od' ,tbss_od , 30, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.3446 - Val Loss: 6.9865
Epoch 2 - Train Loss: 2.1965 - Val Loss: 2.6928
Epoch 3 - Train Loss: 1.7693 - Val Loss: 1.5060
Epoch 4 - Train Loss: 1.8481 - Val Loss: 9.9071
Epoch 5 - Train Loss: 1.7521 - Val Loss: 1.5240
Epoch 6 - Train Loss: 1.5045 - Val Loss: 1.2837
Epoch 7 - Train Loss: 1.6061 - Val Loss: 1.2935
Epoch 8 - Train Loss: 1.3582 - Val Loss: 1.2622
Epoch 9 - Train Loss: 1.4347 - Val Loss: 1.2218
Epoch 10 - Train Loss: 1.3279 - Val Loss: 1.2488
Epoch 11 - Train Loss: 1.4109 - Val Loss: 1.2045
Epoch 12 - Train Loss: 1.3506 - Val Loss: 1.1966
Epoch 13 - Train Loss: 1.3219 - Val Loss: 1.2319
Epoch 14 - Train Loss: 1.1782 - Val Loss: 1.1787
Epoch 15 - Train Loss: 1.4454 - Val Loss: 1.1803
Epoch 16 - Train Loss: 1.2242 - Val Loss: 1.1572
Epoch 17 - Train Loss: 1.2275 - Val Loss: 1.1702
Epoch 18 - Train Loss: 1.0767 - Val Loss: 1.1568
Epoch 19 - Train Loss: 1.0578 - Val Loss: 1.1625
Epoch 20 - Train Loss:

In [22]:
exp2('tbss_rd' ,tbss_rd , 30, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1504 - Val Loss: 1.3268
Epoch 2 - Train Loss: 2.0452 - Val Loss: 6.1111
Epoch 3 - Train Loss: 1.9047 - Val Loss: 8.1663
Epoch 4 - Train Loss: 1.9730 - Val Loss: 2097.8879
Epoch 5 - Train Loss: 1.7190 - Val Loss: 56.0094
Epoch 6 - Train Loss: 1.7150 - Val Loss: 130.1869
Epoch 7 - Train Loss: 1.5660 - Val Loss: 1.4946
Epoch 8 - Train Loss: 1.5830 - Val Loss: 2.0654
Epoch 9 - Train Loss: 1.6001 - Val Loss: 1.3006
Epoch 10 - Train Loss: 1.7151 - Val Loss: 62.1305
Epoch 11 - Train Loss: 1.3056 - Val Loss: 34.2297
Epoch 12 - Train Loss: 1.2236 - Val Loss: 1.2539
Epoch 13 - Train Loss: 1.3637 - Val Loss: 1.1807
Epoch 14 - Train Loss: 1.4658 - Val Loss: 1.2289
Epoch 15 - Train Loss: 1.5763 - Val Loss: 4.8199
Epoch 16 - Train Loss: 1.2418 - Val Loss: 1.1847
Epoch 17 - Train Loss: 1.2140 - Val Loss: 1.2719
Epoch 18 - Train Loss: 1.4869 - Val Loss: 1.1854
Epoch 19 - Train Loss: 1.3350 - Val Loss: 1.4211
Epoch 20 - Tra

In [23]:
exp2('tractography_ad' ,tractography_ad , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1366 - Val Loss: 1.2624
Epoch 2 - Train Loss: 2.1009 - Val Loss: 1.2132
Epoch 3 - Train Loss: 2.0037 - Val Loss: 1.2231
Epoch 4 - Train Loss: 1.7869 - Val Loss: 1.7126
Epoch 5 - Train Loss: 1.4836 - Val Loss: 1.2169
Epoch 6 - Train Loss: 1.7254 - Val Loss: 1.7561
Epoch 7 - Train Loss: 1.4908 - Val Loss: 1.1590
Epoch 8 - Train Loss: 1.6173 - Val Loss: 1.5578
Epoch 9 - Train Loss: 1.3024 - Val Loss: 1.2948
Epoch 10 - Train Loss: 1.3145 - Val Loss: 1.1357
Epoch 11 - Train Loss: 1.3641 - Val Loss: 1.1031
Epoch 12 - Train Loss: 1.3389 - Val Loss: 1.1003
Epoch 13 - Train Loss: 1.2077 - Val Loss: 1.0854
Epoch 14 - Train Loss: 1.6297 - Val Loss: 1.0833
Epoch 15 - Train Loss: 1.3575 - Val Loss: 1.0710
Epoch 16 - Train Loss: 1.3280 - Val Loss: 1.0595
Epoch 17 - Train Loss: 1.3559 - Val Loss: 1.0558
Epoch 18 - Train Loss: 0.9826 - Val Loss: 1.1545
Epoch 19 - Train Loss: 0.9713 - Val Loss: 1.0509
Epoch 20 - Train Loss: 1

In [24]:
exp2('tractography_fa' ,tractography_fa , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.3071 - Val Loss: 2.3730
Epoch 2 - Train Loss: 2.2925 - Val Loss: 9.2165
Epoch 3 - Train Loss: 1.7981 - Val Loss: 1.3262
Epoch 4 - Train Loss: 1.6646 - Val Loss: 1.2597
Epoch 5 - Train Loss: 1.6625 - Val Loss: 1.5863
Epoch 6 - Train Loss: 1.6920 - Val Loss: 1.2063
Epoch 7 - Train Loss: 1.5856 - Val Loss: 1.1706
Epoch 8 - Train Loss: 1.4182 - Val Loss: 1.1936
Epoch 9 - Train Loss: 1.4485 - Val Loss: 1.1495
Epoch 10 - Train Loss: 1.3747 - Val Loss: 1.1284
Epoch 11 - Train Loss: 1.3312 - Val Loss: 1.1200
Epoch 12 - Train Loss: 1.2785 - Val Loss: 1.1183
Epoch 13 - Train Loss: 1.2510 - Val Loss: 1.1086
Epoch 14 - Train Loss: 1.3102 - Val Loss: 1.0930
Epoch 15 - Train Loss: 1.1876 - Val Loss: 1.0878
Epoch 16 - Train Loss: 1.0972 - Val Loss: 1.0879
Epoch 17 - Train Loss: 1.1705 - Val Loss: 1.0735
Epoch 18 - Train Loss: 1.1742 - Val Loss: 1.0748
Epoch 19 - Train Loss: 1.0311 - Val Loss: 1.0674
Epoch 20 - Train Loss: 0

In [25]:
exp2('tractography_icvf' ,tractography_icvf , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.3684 - Val Loss: 11.2252
Epoch 2 - Train Loss: 2.1151 - Val Loss: 2.9756
Epoch 3 - Train Loss: 1.9115 - Val Loss: 350.5326
Epoch 4 - Train Loss: 1.7355 - Val Loss: 1.3027
Epoch 5 - Train Loss: 1.5565 - Val Loss: 1.2870
Epoch 6 - Train Loss: 1.5376 - Val Loss: 1.2295
Epoch 7 - Train Loss: 1.5608 - Val Loss: 1.2234
Epoch 8 - Train Loss: 1.4152 - Val Loss: 1.1969
Epoch 9 - Train Loss: 1.3243 - Val Loss: 1.2020
Epoch 10 - Train Loss: 1.5096 - Val Loss: 1.1818
Epoch 11 - Train Loss: 1.5037 - Val Loss: 1.1704
Epoch 12 - Train Loss: 1.4409 - Val Loss: 1.1584
Epoch 13 - Train Loss: 1.3924 - Val Loss: 1.1359
Epoch 14 - Train Loss: 1.5640 - Val Loss: 1.1302
Epoch 15 - Train Loss: 1.2578 - Val Loss: 1.1319
Epoch 16 - Train Loss: 1.2388 - Val Loss: 1.1281
Epoch 17 - Train Loss: 1.2090 - Val Loss: 1.1197
Epoch 18 - Train Loss: 1.1796 - Val Loss: 1.1088
Epoch 19 - Train Loss: 1.1606 - Val Loss: 1.1028
Epoch 20 - Train Loss

In [26]:
exp2('tractography_isovf' ,tractography_isovf , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.4435 - Val Loss: 1.3744
Epoch 2 - Train Loss: 1.9071 - Val Loss: 2.2290
Epoch 3 - Train Loss: 1.7327 - Val Loss: 1.3270
Epoch 4 - Train Loss: 1.7345 - Val Loss: 1.2644
Epoch 5 - Train Loss: 1.6976 - Val Loss: 1.2485
Epoch 6 - Train Loss: 1.9310 - Val Loss: 1.2164
Epoch 7 - Train Loss: 1.4492 - Val Loss: 1.2026
Epoch 8 - Train Loss: 1.4173 - Val Loss: 1.1682
Epoch 9 - Train Loss: 1.3280 - Val Loss: 1.1640
Epoch 10 - Train Loss: 1.3316 - Val Loss: 2.5677
Epoch 11 - Train Loss: 1.4756 - Val Loss: 1.1399
Epoch 12 - Train Loss: 1.1266 - Val Loss: 1.1476
Epoch 13 - Train Loss: 1.2931 - Val Loss: 1.1301
Epoch 14 - Train Loss: 1.2990 - Val Loss: 1.1213
Epoch 15 - Train Loss: 1.1956 - Val Loss: 1.1068
Epoch 16 - Train Loss: 1.2890 - Val Loss: 1.1074
Epoch 17 - Train Loss: 1.2354 - Val Loss: 1.0982
Epoch 18 - Train Loss: 1.1585 - Val Loss: 1.0988
Epoch 19 - Train Loss: 1.1430 - Val Loss: 1.0959
Epoch 20 - Train Loss: 1

In [27]:
exp2('tractography_md' ,tractography_md , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2656 - Val Loss: 1.3587
Epoch 2 - Train Loss: 1.8885 - Val Loss: 1.3791
Epoch 3 - Train Loss: 1.8061 - Val Loss: 4.5689
Epoch 4 - Train Loss: 1.6726 - Val Loss: 3.8129
Epoch 5 - Train Loss: 1.6048 - Val Loss: 1.7330
Epoch 6 - Train Loss: 1.7082 - Val Loss: 7.0424
Epoch 7 - Train Loss: 1.4440 - Val Loss: 1.5179
Epoch 8 - Train Loss: 1.4800 - Val Loss: 1.2648
Epoch 9 - Train Loss: 1.2481 - Val Loss: 1.2400
Epoch 10 - Train Loss: 1.4135 - Val Loss: 1.2844
Epoch 11 - Train Loss: 1.1839 - Val Loss: 1.6372
Epoch 12 - Train Loss: 1.3135 - Val Loss: 1.2058
Epoch 13 - Train Loss: 1.2033 - Val Loss: 1.4588
Epoch 14 - Train Loss: 1.2909 - Val Loss: 1.2684
Epoch 15 - Train Loss: 1.2216 - Val Loss: 1.1896
Epoch 16 - Train Loss: 1.2770 - Val Loss: 1.1839
Epoch 17 - Train Loss: 1.3027 - Val Loss: 1.1873
Epoch 18 - Train Loss: 1.1467 - Val Loss: 1.1796
Epoch 19 - Train Loss: 1.1889 - Val Loss: 1.1849
Epoch 20 - Train Loss: 1

In [28]:
exp2('tractography_od' ,tractography_od , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2879 - Val Loss: 1.5813
Epoch 2 - Train Loss: 1.9862 - Val Loss: 2.2903
Epoch 3 - Train Loss: 1.6247 - Val Loss: 1.5166
Epoch 4 - Train Loss: 1.8326 - Val Loss: 1.3781
Epoch 5 - Train Loss: 1.7191 - Val Loss: 1.3337
Epoch 6 - Train Loss: 1.6026 - Val Loss: 1.2848
Epoch 7 - Train Loss: 1.5927 - Val Loss: 1.2616
Epoch 8 - Train Loss: 1.4257 - Val Loss: 1.2517
Epoch 9 - Train Loss: 1.3991 - Val Loss: 1.2206
Epoch 10 - Train Loss: 1.3534 - Val Loss: 1.2200
Epoch 11 - Train Loss: 1.2279 - Val Loss: 1.1967
Epoch 12 - Train Loss: 1.2320 - Val Loss: 1.1966
Epoch 13 - Train Loss: 1.2840 - Val Loss: 128.1290
Epoch 14 - Train Loss: 1.2573 - Val Loss: 1.1633
Epoch 15 - Train Loss: 1.2721 - Val Loss: 1.1661
Epoch 16 - Train Loss: 1.1667 - Val Loss: 1.1509
Epoch 17 - Train Loss: 1.1578 - Val Loss: 1.1615
Epoch 18 - Train Loss: 1.3392 - Val Loss: 1.1538
Epoch 19 - Train Loss: 1.3035 - Val Loss: 1.1469
Epoch 20 - Train Loss:

In [29]:
exp2('tractography_rd' ,tractography_rd , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2831 - Val Loss: 1.3390
Epoch 2 - Train Loss: 2.1768 - Val Loss: 1.3362
Epoch 3 - Train Loss: 2.0311 - Val Loss: 1.5626
Epoch 4 - Train Loss: 1.9321 - Val Loss: 130.5599
Epoch 5 - Train Loss: 1.5869 - Val Loss: 31.8284
Epoch 6 - Train Loss: 1.4311 - Val Loss: 2.1208
Epoch 7 - Train Loss: 1.4464 - Val Loss: 1.2532
Epoch 8 - Train Loss: 1.4158 - Val Loss: 1.2543
Epoch 9 - Train Loss: 1.3083 - Val Loss: 1.4740
Epoch 10 - Train Loss: 1.2236 - Val Loss: 1.2086
Epoch 11 - Train Loss: 1.7389 - Val Loss: 2.4284
Epoch 12 - Train Loss: 1.1956 - Val Loss: 1.4135
Epoch 13 - Train Loss: 1.2631 - Val Loss: 1.1931
Epoch 14 - Train Loss: 1.2765 - Val Loss: 4.2567
Epoch 15 - Train Loss: 1.2945 - Val Loss: 1.1472
Epoch 16 - Train Loss: 1.2403 - Val Loss: 1.1359
Epoch 17 - Train Loss: 1.1771 - Val Loss: 1.1877
Epoch 18 - Train Loss: 1.2380 - Val Loss: 1.1386
Epoch 19 - Train Loss: 1.2923 - Val Loss: 1.1455
Epoch 20 - Train Loss

In [30]:
exp2('tractography_fss' ,tractography_fss, 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 1.8745 - Val Loss: 2.6728
Epoch 2 - Train Loss: 2.1364 - Val Loss: 1.5531
Epoch 3 - Train Loss: 2.0682 - Val Loss: 1.4333
Epoch 4 - Train Loss: 1.8531 - Val Loss: 1.3929
Epoch 5 - Train Loss: 1.7184 - Val Loss: 1.4049
Epoch 6 - Train Loss: 1.6234 - Val Loss: 1.3535
Epoch 7 - Train Loss: 1.5658 - Val Loss: 1.3333
Epoch 8 - Train Loss: 1.8155 - Val Loss: 1.3403
Epoch 9 - Train Loss: 1.6299 - Val Loss: 1.3143
Epoch 10 - Train Loss: 1.3813 - Val Loss: 1.3241
Epoch 11 - Train Loss: 1.2188 - Val Loss: 1.2842
Epoch 12 - Train Loss: 1.3322 - Val Loss: 1.2828
Epoch 13 - Train Loss: 1.3041 - Val Loss: 1.2794
Epoch 14 - Train Loss: 1.2789 - Val Loss: 1.2726
Epoch 15 - Train Loss: 1.2294 - Val Loss: 1.3186
Epoch 16 - Train Loss: 1.2761 - Val Loss: 1.2642
Epoch 17 - Train Loss: 1.2119 - Val Loss: 1.2646
Epoch 18 - Train Loss: 1.1369 - Val Loss: 1.2518
Epoch 19 - Train Loss: 1.0929 - Val Loss: 1.2430
Epoch 20 - Train Loss: 1

In [31]:
exp2('tractography_fssl' ,tractography_fssl, 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 1.9496 - Val Loss: 12.4366
Epoch 2 - Train Loss: 1.8147 - Val Loss: 24.2390
Epoch 3 - Train Loss: 1.8943 - Val Loss: 1.3942
Epoch 4 - Train Loss: 1.7392 - Val Loss: 1.3033
Epoch 5 - Train Loss: 1.6855 - Val Loss: 1.2779
Epoch 6 - Train Loss: 1.6596 - Val Loss: 1.2460
Epoch 7 - Train Loss: 1.8255 - Val Loss: 1.2015
Epoch 8 - Train Loss: 1.5230 - Val Loss: 1.2194
Epoch 9 - Train Loss: 1.5447 - Val Loss: 1.1901
Epoch 10 - Train Loss: 1.4567 - Val Loss: 1.1848
Epoch 11 - Train Loss: 1.4024 - Val Loss: 1.1861
Epoch 12 - Train Loss: 1.5654 - Val Loss: 1.1584
Epoch 13 - Train Loss: 1.2732 - Val Loss: 1.1404
Epoch 14 - Train Loss: 1.3004 - Val Loss: 1.1418
Epoch 15 - Train Loss: 1.3664 - Val Loss: 1.1299
Epoch 16 - Train Loss: 1.4177 - Val Loss: 1.1616
Epoch 17 - Train Loss: 1.1021 - Val Loss: 1.1171
Epoch 18 - Train Loss: 1.3525 - Val Loss: 1.1036
Epoch 19 - Train Loss: 1.1725 - Val Loss: 1.1004
Epoch 20 - Train Loss:

In [32]:
exp2('f_gamb' ,f_gamb , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 1.8329 - Val Loss: 1.7379
Epoch 2 - Train Loss: 2.0602 - Val Loss: 1.3422
Epoch 3 - Train Loss: 1.8759 - Val Loss: 1.2935
Epoch 4 - Train Loss: 1.6857 - Val Loss: 1.3127
Epoch 5 - Train Loss: 1.6375 - Val Loss: 1.2480
Epoch 6 - Train Loss: 1.6320 - Val Loss: 1.2236
Epoch 7 - Train Loss: 1.5133 - Val Loss: 1.2099
Epoch 8 - Train Loss: 1.6071 - Val Loss: 1.1900
Epoch 9 - Train Loss: 1.3875 - Val Loss: 1.1871
Epoch 10 - Train Loss: 1.3425 - Val Loss: 1.1656
Epoch 11 - Train Loss: 1.3034 - Val Loss: 1.1683
Epoch 12 - Train Loss: 1.4511 - Val Loss: 5.7529
Epoch 13 - Train Loss: 1.2885 - Val Loss: 1.1491
Epoch 14 - Train Loss: 1.3690 - Val Loss: 1.1317
Epoch 15 - Train Loss: 1.2062 - Val Loss: 1.1286
Epoch 16 - Train Loss: 1.3617 - Val Loss: 1.1196
Epoch 17 - Train Loss: 1.2489 - Val Loss: 1.1182
Epoch 18 - Train Loss: 1.1861 - Val Loss: 1.1111
Epoch 19 - Train Loss: 1.2313 - Val Loss: 1.1109
Epoch 20 - Train Loss: 1

In [33]:
exp2('f_lang' ,f_lang , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2369 - Val Loss: 2.2270
Epoch 2 - Train Loss: 1.8383 - Val Loss: 1.3850
Epoch 3 - Train Loss: 1.9339 - Val Loss: 1.3410
Epoch 4 - Train Loss: 1.7932 - Val Loss: 1.3147
Epoch 5 - Train Loss: 1.6447 - Val Loss: 1.3822
Epoch 6 - Train Loss: 1.9706 - Val Loss: 1.2461
Epoch 7 - Train Loss: 1.5956 - Val Loss: 1.1838
Epoch 8 - Train Loss: 1.4312 - Val Loss: 1.2021
Epoch 9 - Train Loss: 1.3423 - Val Loss: 1.1616
Epoch 10 - Train Loss: 1.4093 - Val Loss: 1.1676
Epoch 11 - Train Loss: 1.3140 - Val Loss: 1.1303
Epoch 12 - Train Loss: 1.3693 - Val Loss: 1.1167
Epoch 13 - Train Loss: 1.2736 - Val Loss: 1.0945
Epoch 14 - Train Loss: 1.5582 - Val Loss: 1.0928
Epoch 15 - Train Loss: 1.4708 - Val Loss: 1.0727
Epoch 16 - Train Loss: 1.1230 - Val Loss: 1.0633
Epoch 17 - Train Loss: 1.1446 - Val Loss: 1.0583
Epoch 18 - Train Loss: 1.2572 - Val Loss: 1.0599
Epoch 19 - Train Loss: 1.1481 - Val Loss: 1.0453
Epoch 20 - Train Loss: 1

In [34]:
exp2('f_motor' ,f_motor , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.4545 - Val Loss: 1.4860
Epoch 2 - Train Loss: 2.2264 - Val Loss: 5.1768
Epoch 3 - Train Loss: 1.7537 - Val Loss: 1.3673
Epoch 4 - Train Loss: 1.7174 - Val Loss: 1.3838
Epoch 5 - Train Loss: 1.6265 - Val Loss: 1.2784
Epoch 6 - Train Loss: 1.6928 - Val Loss: 1.2712
Epoch 7 - Train Loss: 1.8357 - Val Loss: 1.2454
Epoch 8 - Train Loss: 1.3923 - Val Loss: 1.2209
Epoch 9 - Train Loss: 1.3765 - Val Loss: 1.2113
Epoch 10 - Train Loss: 1.3490 - Val Loss: 1.1935
Epoch 11 - Train Loss: 1.3702 - Val Loss: 1.1927
Epoch 12 - Train Loss: 1.2402 - Val Loss: 1.1834
Epoch 13 - Train Loss: 1.2880 - Val Loss: 1.1706
Epoch 14 - Train Loss: 1.3344 - Val Loss: 1.1637
Epoch 15 - Train Loss: 1.2308 - Val Loss: 1.1668
Epoch 16 - Train Loss: 1.2492 - Val Loss: 1.1538
Epoch 17 - Train Loss: 1.0577 - Val Loss: 1.1536
Epoch 18 - Train Loss: 1.1423 - Val Loss: 1.1449
Epoch 19 - Train Loss: 1.2612 - Val Loss: 1.1460
Epoch 20 - Train Loss: 1

In [35]:
exp2('f_rs' ,f_rs , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2605 - Val Loss: 1.6444
Epoch 2 - Train Loss: 1.9774 - Val Loss: 1.5230
Epoch 3 - Train Loss: 1.6883 - Val Loss: 1.3749
Epoch 4 - Train Loss: 1.7688 - Val Loss: 1.3720
Epoch 5 - Train Loss: 1.5232 - Val Loss: 1.3137
Epoch 6 - Train Loss: 1.5775 - Val Loss: 1.2870
Epoch 7 - Train Loss: 1.4312 - Val Loss: 1.2361
Epoch 8 - Train Loss: 1.4648 - Val Loss: 1.2109
Epoch 9 - Train Loss: 1.3984 - Val Loss: 439.0586
Epoch 10 - Train Loss: 1.4404 - Val Loss: 1.1854
Epoch 11 - Train Loss: 1.2869 - Val Loss: 1.1601
Epoch 12 - Train Loss: 1.1676 - Val Loss: 1.2312
Epoch 13 - Train Loss: 1.2441 - Val Loss: 1.1396
Epoch 14 - Train Loss: 1.5190 - Val Loss: 1.1464
Epoch 15 - Train Loss: 1.2850 - Val Loss: 1.1315
Epoch 16 - Train Loss: 1.2877 - Val Loss: 1.1192
Epoch 17 - Train Loss: 1.4777 - Val Loss: 1.1145
Epoch 18 - Train Loss: 1.1155 - Val Loss: 1.1169
Epoch 19 - Train Loss: 1.1969 - Val Loss: 1.1109
Epoch 20 - Train Loss:

In [36]:
exp2('f_social' ,f_social , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1799 - Val Loss: 1.6869
Epoch 2 - Train Loss: 1.9102 - Val Loss: 1.4387
Epoch 3 - Train Loss: 1.8878 - Val Loss: 1.3141
Epoch 4 - Train Loss: 1.5923 - Val Loss: 1.2777
Epoch 5 - Train Loss: 1.5520 - Val Loss: 1.2402
Epoch 6 - Train Loss: 1.6623 - Val Loss: 1.2014
Epoch 7 - Train Loss: 1.6715 - Val Loss: 1.1828
Epoch 8 - Train Loss: 1.5290 - Val Loss: 1.1655
Epoch 9 - Train Loss: 1.4444 - Val Loss: 1.1433
Epoch 10 - Train Loss: 1.5256 - Val Loss: 1.1476
Epoch 11 - Train Loss: 1.4526 - Val Loss: 1.1498
Epoch 12 - Train Loss: 1.3563 - Val Loss: 1.1498
Epoch 13 - Train Loss: 1.4338 - Val Loss: 1.1172
Epoch 14 - Train Loss: 1.3118 - Val Loss: 1.1157
Epoch 15 - Train Loss: 1.2362 - Val Loss: 1.0970
Epoch 16 - Train Loss: 1.1871 - Val Loss: 1.1177
Epoch 17 - Train Loss: 1.1272 - Val Loss: 1.0882
Epoch 18 - Train Loss: 1.1151 - Val Loss: 1.0774
Epoch 19 - Train Loss: 1.1972 - Val Loss: 1.0777
Epoch 20 - Train Loss: 1

In [37]:
exp2('f_wm' ,f_wm , 30, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.3137 - Val Loss: 2.0375
Epoch 2 - Train Loss: 2.0216 - Val Loss: 1.9617
Epoch 3 - Train Loss: 1.8923 - Val Loss: 1.4006
Epoch 4 - Train Loss: 1.5799 - Val Loss: 1.4122
Epoch 5 - Train Loss: 1.5657 - Val Loss: 1.3773
Epoch 6 - Train Loss: 1.4837 - Val Loss: 2.3045
Epoch 7 - Train Loss: 1.4929 - Val Loss: 1.3315
Epoch 8 - Train Loss: 1.3938 - Val Loss: 1.3022
Epoch 9 - Train Loss: 1.3027 - Val Loss: 1.2601
Epoch 10 - Train Loss: 1.4642 - Val Loss: 1.2371
Epoch 11 - Train Loss: 1.3752 - Val Loss: 1.2364
Epoch 12 - Train Loss: 1.3330 - Val Loss: 1.1951
Epoch 13 - Train Loss: 1.2139 - Val Loss: 1.2190
Epoch 14 - Train Loss: 1.5688 - Val Loss: 1.2003
Epoch 15 - Train Loss: 1.2329 - Val Loss: 1.1886
Epoch 16 - Train Loss: 1.0687 - Val Loss: 1.1668
Epoch 17 - Train Loss: 1.3411 - Val Loss: 1.1722
Epoch 18 - Train Loss: 1.2479 - Val Loss: 1.1756
Epoch 19 - Train Loss: 1.1286 - Val Loss: 1.1665
Epoch 20 - Train Loss: 1

In [66]:
exp('cort_thick' ,cort_thick, 50, 10)

iter  1
(602, 400) (151, 400) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.4490 - Val Loss: 1.4817
Epoch 2 - Train Loss: 2.4142 - Val Loss: 2.7534
Epoch 3 - Train Loss: 2.1795 - Val Loss: 1.4361
Epoch 4 - Train Loss: 2.1474 - Val Loss: 1.3895
Epoch 5 - Train Loss: 1.9107 - Val Loss: 1.5502
Epoch 6 - Train Loss: 2.0941 - Val Loss: 1.3226
Epoch 7 - Train Loss: 1.9266 - Val Loss: 1.2898
Epoch 8 - Train Loss: 1.9833 - Val Loss: 1.3206
Epoch 9 - Train Loss: 1.7173 - Val Loss: 1.2832
Epoch 10 - Train Loss: 1.7923 - Val Loss: 1.2549
Epoch 11 - Train Loss: 1.7080 - Val Loss: 1.2639
Epoch 12 - Train Loss: 1.6860 - Val Loss: 1.2455
Epoch 13 - Train Loss: 1.6230 - Val Loss: 1.2390
Epoch 14 - Train Loss: 1.5523 - Val Loss: 1.2116
Epoch 15 - Train Loss: 2.0838 - Val Loss: 1.2126
Epoch 16 - Train Loss: 1.3680 - Val Loss: 1.2269
Epoch 17 - Train Loss: 1.6650 - Val Loss: 1.2182
Epoch 18 - Train Loss: 1.5930 - Val Loss: 1.2099
Epoch 19 - Train Loss: 1.4125 - Val Loss: 1.1977
Epoch 20 - Train Loss: 1.288

In [67]:
exp('cort_area' ,cort_area, 50, 10)

iter  1
(602, 400) (151, 400) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2693 - Val Loss: 1.3758
Epoch 2 - Train Loss: 2.3044 - Val Loss: 1.3007
Epoch 3 - Train Loss: 1.9887 - Val Loss: 1.2935
Epoch 4 - Train Loss: 1.8511 - Val Loss: 1.2665
Epoch 5 - Train Loss: 1.8708 - Val Loss: 1.2372
Epoch 6 - Train Loss: 1.8817 - Val Loss: 1.2338
Epoch 7 - Train Loss: 1.7800 - Val Loss: 1.2886
Epoch 8 - Train Loss: 1.8387 - Val Loss: 1.2241
Epoch 9 - Train Loss: 1.9774 - Val Loss: 1.2071
Epoch 10 - Train Loss: 1.7116 - Val Loss: 1.1933
Epoch 11 - Train Loss: 1.7724 - Val Loss: 1.2086
Epoch 12 - Train Loss: 1.4889 - Val Loss: 1.2020
Epoch 13 - Train Loss: 1.7362 - Val Loss: 1.1775
Epoch 14 - Train Loss: 1.5720 - Val Loss: 1.1740
Epoch 15 - Train Loss: 1.6333 - Val Loss: 1.1681
Epoch 16 - Train Loss: 1.5010 - Val Loss: 1.1656
Epoch 17 - Train Loss: 1.6570 - Val Loss: 1.1520
Epoch 18 - Train Loss: 1.3873 - Val Loss: 1.1361
Epoch 19 - Train Loss: 1.3419 - Val Loss: 1.1335
Epoch 20 - Train Loss: 1.318

In [68]:
exp('cort_vol' ,cort_vol, 50, 10)

iter  1
(602, 400) (151, 400) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.0295 - Val Loss: 1.3055
Epoch 2 - Train Loss: 2.1621 - Val Loss: 1.3385
Epoch 3 - Train Loss: 2.0556 - Val Loss: 1.3075
Epoch 4 - Train Loss: 1.7782 - Val Loss: 1.2819
Epoch 5 - Train Loss: 1.8946 - Val Loss: 1.2648
Epoch 6 - Train Loss: 1.8738 - Val Loss: 1.2295
Epoch 7 - Train Loss: 1.8525 - Val Loss: 1.2188
Epoch 8 - Train Loss: 2.1043 - Val Loss: 1.2154
Epoch 9 - Train Loss: 1.8114 - Val Loss: 1.1899
Epoch 10 - Train Loss: 1.6621 - Val Loss: 1.1791
Epoch 11 - Train Loss: 1.7024 - Val Loss: 1.1768
Epoch 12 - Train Loss: 1.5989 - Val Loss: 1.1652
Epoch 13 - Train Loss: 1.4998 - Val Loss: 1.1488
Epoch 14 - Train Loss: 1.4874 - Val Loss: 1.1431
Epoch 15 - Train Loss: 1.5229 - Val Loss: 1.1416
Epoch 16 - Train Loss: 1.5196 - Val Loss: 1.1316
Epoch 17 - Train Loss: 1.6413 - Val Loss: 1.1393
Epoch 18 - Train Loss: 1.5892 - Val Loss: 1.1173
Epoch 19 - Train Loss: 1.5788 - Val Loss: 1.1141
Epoch 20 - Train Loss: 1.360

In [38]:
exp2('tbss_ad' ,tbss_ad, 50, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2535 - Val Loss: 1.3502
Epoch 2 - Train Loss: 2.0061 - Val Loss: 1.3789
Epoch 3 - Train Loss: 1.9321 - Val Loss: 6.1839
Epoch 4 - Train Loss: 1.9323 - Val Loss: 4.1589
Epoch 5 - Train Loss: 1.7521 - Val Loss: 17.3109
Epoch 6 - Train Loss: 1.7941 - Val Loss: 4.4180
Epoch 7 - Train Loss: 1.9388 - Val Loss: 1.7380
Epoch 8 - Train Loss: 1.3594 - Val Loss: 1.5405
Epoch 9 - Train Loss: 1.5703 - Val Loss: 1.3283
Epoch 10 - Train Loss: 1.7079 - Val Loss: 4.0903
Epoch 11 - Train Loss: 1.4790 - Val Loss: 1.2702
Epoch 12 - Train Loss: 1.3717 - Val Loss: 1.2112
Epoch 13 - Train Loss: 1.4038 - Val Loss: 1.1899
Epoch 14 - Train Loss: 1.2890 - Val Loss: 1.1901
Epoch 15 - Train Loss: 1.2787 - Val Loss: 1.1659
Epoch 16 - Train Loss: 1.4729 - Val Loss: 1.1467
Epoch 17 - Train Loss: 1.2230 - Val Loss: 1.1908
Epoch 18 - Train Loss: 1.4467 - Val Loss: 1.1279
Epoch 19 - Train Loss: 1.5946 - Val Loss: 1.1257
Epoch 20 - Train Loss

In [39]:
exp2('tbss_fa' ,tbss_fa , 50, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2468 - Val Loss: 4.2391
Epoch 2 - Train Loss: 2.1175 - Val Loss: 4.8443
Epoch 3 - Train Loss: 1.9791 - Val Loss: 2.2030
Epoch 4 - Train Loss: 1.8491 - Val Loss: 2.3556
Epoch 5 - Train Loss: 1.7339 - Val Loss: 1.6682
Epoch 6 - Train Loss: 1.7541 - Val Loss: 1.8756
Epoch 7 - Train Loss: 1.8310 - Val Loss: 3.5093
Epoch 8 - Train Loss: 1.6042 - Val Loss: 1.2924
Epoch 9 - Train Loss: 1.6129 - Val Loss: 1.2406
Epoch 10 - Train Loss: 1.4374 - Val Loss: 1.3715
Epoch 11 - Train Loss: 1.7018 - Val Loss: 1.1970
Epoch 12 - Train Loss: 1.5532 - Val Loss: 1.1778
Epoch 13 - Train Loss: 1.3438 - Val Loss: 1.3041
Epoch 14 - Train Loss: 1.5146 - Val Loss: 1.1775
Epoch 15 - Train Loss: 1.3663 - Val Loss: 1.2373
Epoch 16 - Train Loss: 1.2999 - Val Loss: 1.1306
Epoch 17 - Train Loss: 1.3083 - Val Loss: 1.1270
Epoch 18 - Train Loss: 1.2242 - Val Loss: 1.1155
Epoch 19 - Train Loss: 1.2792 - Val Loss: 1.1013
Epoch 20 - Train Loss:

In [40]:
exp2('tbss_icvf' ,tbss_icvf , 50, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1237 - Val Loss: 5.8725
Epoch 2 - Train Loss: 2.1489 - Val Loss: 3.2418
Epoch 3 - Train Loss: 2.0570 - Val Loss: 3.3685
Epoch 4 - Train Loss: 1.9364 - Val Loss: 7.0030
Epoch 5 - Train Loss: 1.5557 - Val Loss: 1.5751
Epoch 6 - Train Loss: 1.6703 - Val Loss: 1.5729
Epoch 7 - Train Loss: 1.6312 - Val Loss: 1.4422
Epoch 8 - Train Loss: 1.5914 - Val Loss: 1.3103
Epoch 9 - Train Loss: 1.4983 - Val Loss: 1.3300
Epoch 10 - Train Loss: 1.4783 - Val Loss: 1.2576
Epoch 11 - Train Loss: 1.6091 - Val Loss: 1.2787
Epoch 12 - Train Loss: 1.3728 - Val Loss: 1.2051
Epoch 13 - Train Loss: 1.2341 - Val Loss: 1.2013
Epoch 14 - Train Loss: 1.2547 - Val Loss: 1.1917
Epoch 15 - Train Loss: 1.2371 - Val Loss: 1.1934
Epoch 16 - Train Loss: 1.2078 - Val Loss: 1.2221
Epoch 17 - Train Loss: 1.2053 - Val Loss: 1.1703
Epoch 18 - Train Loss: 1.4409 - Val Loss: 1.1766
Epoch 19 - Train Loss: 1.3593 - Val Loss: 1.1643
Epoch 20 - Train Loss:

In [41]:
exp2('tbss_isovf' ,tbss_isovf, 50, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2838 - Val Loss: 2.0302
Epoch 2 - Train Loss: 1.9465 - Val Loss: 1.5310
Epoch 3 - Train Loss: 1.7690 - Val Loss: 1.4336
Epoch 4 - Train Loss: 1.6329 - Val Loss: 1.3079
Epoch 5 - Train Loss: 1.4571 - Val Loss: 1.2501
Epoch 6 - Train Loss: 1.5164 - Val Loss: 1.2535
Epoch 7 - Train Loss: 1.4978 - Val Loss: 1.2183
Epoch 8 - Train Loss: 1.5359 - Val Loss: 1.2023
Epoch 9 - Train Loss: 1.4066 - Val Loss: 1.1848
Epoch 10 - Train Loss: 1.3741 - Val Loss: 1.1671
Epoch 11 - Train Loss: 1.2276 - Val Loss: 1.1591
Epoch 12 - Train Loss: 1.4030 - Val Loss: 1.1441
Epoch 13 - Train Loss: 1.2676 - Val Loss: 1.1417
Epoch 14 - Train Loss: 1.2841 - Val Loss: 1.1312
Epoch 15 - Train Loss: 1.0455 - Val Loss: 1.1312
Epoch 16 - Train Loss: 1.1899 - Val Loss: 1.1233
Epoch 17 - Train Loss: 1.1660 - Val Loss: 1.1188
Epoch 18 - Train Loss: 1.2682 - Val Loss: 1.1185
Epoch 19 - Train Loss: 1.2024 - Val Loss: 1.1093
Epoch 20 - Train Loss:

In [42]:
exp2('tbss_md' ,tbss_md , 50, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2397 - Val Loss: 1.3975
Epoch 2 - Train Loss: 2.0895 - Val Loss: 1.9838
Epoch 3 - Train Loss: 1.9248 - Val Loss: 8.7361
Epoch 4 - Train Loss: 2.0545 - Val Loss: 71.4517
Epoch 5 - Train Loss: 1.7858 - Val Loss: 333.6037
Epoch 6 - Train Loss: 1.8762 - Val Loss: 66.1449
Epoch 7 - Train Loss: 1.8407 - Val Loss: 6.3568
Epoch 8 - Train Loss: 1.7835 - Val Loss: 3.5842
Epoch 9 - Train Loss: 1.7453 - Val Loss: 9.6046
Epoch 10 - Train Loss: 1.8294 - Val Loss: 2.1330
Epoch 11 - Train Loss: 1.4474 - Val Loss: 1.2881
Epoch 12 - Train Loss: 1.5687 - Val Loss: 2.5183
Epoch 13 - Train Loss: 1.5948 - Val Loss: 1.2886
Epoch 14 - Train Loss: 1.7033 - Val Loss: 1.3355
Epoch 15 - Train Loss: 1.4916 - Val Loss: 1.9706
Epoch 16 - Train Loss: 1.2843 - Val Loss: 1.1997
Epoch 17 - Train Loss: 1.4882 - Val Loss: 1.2163
Epoch 18 - Train Loss: 1.3685 - Val Loss: 3.2871
Epoch 19 - Train Loss: 1.3434 - Val Loss: 1.1982
Epoch 20 - Train L

In [43]:
exp2('tbss_od' ,tbss_od , 50, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.0646 - Val Loss: 5.9600
Epoch 2 - Train Loss: 2.0219 - Val Loss: 4.0504
Epoch 3 - Train Loss: 1.7586 - Val Loss: 1.2515
Epoch 4 - Train Loss: 1.7828 - Val Loss: 1.6744
Epoch 5 - Train Loss: 1.6094 - Val Loss: 1.2397
Epoch 6 - Train Loss: 1.6357 - Val Loss: 1.9428
Epoch 7 - Train Loss: 1.4990 - Val Loss: 1.1415
Epoch 8 - Train Loss: 1.3028 - Val Loss: 7.1800
Epoch 9 - Train Loss: 1.4103 - Val Loss: 1.1625
Epoch 10 - Train Loss: 1.3105 - Val Loss: 1.1432
Epoch 11 - Train Loss: 1.3258 - Val Loss: 3.3228
Epoch 12 - Train Loss: 1.2113 - Val Loss: 1.1161
Epoch 13 - Train Loss: 1.3049 - Val Loss: 1.0877
Epoch 14 - Train Loss: 1.2481 - Val Loss: 1.0767
Epoch 15 - Train Loss: 1.1588 - Val Loss: 1.0804
Epoch 16 - Train Loss: 1.2148 - Val Loss: 1.0743
Epoch 17 - Train Loss: 1.2213 - Val Loss: 1.0576
Epoch 18 - Train Loss: 1.1057 - Val Loss: 1.0595
Epoch 19 - Train Loss: 1.1150 - Val Loss: 1.0505
Epoch 20 - Train Loss:

In [44]:
exp2('tbss_rd' ,tbss_rd , 50, 10)

iter  1
(602, 133053) (151, 133053) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1436 - Val Loss: 1.3223
Epoch 2 - Train Loss: 2.0960 - Val Loss: 2.8823
Epoch 3 - Train Loss: 2.0405 - Val Loss: 86.0566
Epoch 4 - Train Loss: 2.0142 - Val Loss: 198.4671
Epoch 5 - Train Loss: 1.7347 - Val Loss: 194.2325
Epoch 6 - Train Loss: 1.7174 - Val Loss: 4.9267
Epoch 7 - Train Loss: 1.4775 - Val Loss: 55.3847
Epoch 8 - Train Loss: 1.4062 - Val Loss: 3.0406
Epoch 9 - Train Loss: 1.5532 - Val Loss: 1.4565
Epoch 10 - Train Loss: 1.6211 - Val Loss: 1.4799
Epoch 11 - Train Loss: 1.4373 - Val Loss: 5.8536
Epoch 12 - Train Loss: 1.4854 - Val Loss: 1.3344
Epoch 13 - Train Loss: 1.3713 - Val Loss: 1.6886
Epoch 14 - Train Loss: 1.3543 - Val Loss: 1.3445
Epoch 15 - Train Loss: 1.1702 - Val Loss: 1.2295
Epoch 16 - Train Loss: 1.1427 - Val Loss: 1.2241
Epoch 17 - Train Loss: 1.3554 - Val Loss: 1.2048
Epoch 18 - Train Loss: 1.3449 - Val Loss: 1.2343
Epoch 19 - Train Loss: 1.0765 - Val Loss: 1.3126
Epoch 20 - Train

In [45]:
exp2('tractography_ad' ,tractography_ad , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1818 - Val Loss: 1.3233
Epoch 2 - Train Loss: 2.0663 - Val Loss: 1.2781
Epoch 3 - Train Loss: 1.8408 - Val Loss: 2.3044
Epoch 4 - Train Loss: 1.5655 - Val Loss: 1.3617
Epoch 5 - Train Loss: 1.7233 - Val Loss: 1.2837
Epoch 6 - Train Loss: 1.3631 - Val Loss: 1.3664
Epoch 7 - Train Loss: 1.6066 - Val Loss: 1.2142
Epoch 8 - Train Loss: 1.3414 - Val Loss: 1.2115
Epoch 9 - Train Loss: 1.4121 - Val Loss: 1.2003
Epoch 10 - Train Loss: 1.4116 - Val Loss: 1.1716
Epoch 11 - Train Loss: 1.4268 - Val Loss: 1.1914
Epoch 12 - Train Loss: 1.2654 - Val Loss: 1.1527
Epoch 13 - Train Loss: 1.4291 - Val Loss: 1.1384
Epoch 14 - Train Loss: 1.1282 - Val Loss: 1.1344
Epoch 15 - Train Loss: 1.2571 - Val Loss: 1.1663
Epoch 16 - Train Loss: 1.1821 - Val Loss: 1.1193
Epoch 17 - Train Loss: 1.2796 - Val Loss: 1.1200
Epoch 18 - Train Loss: 1.2847 - Val Loss: 1.1223
Epoch 19 - Train Loss: 1.1976 - Val Loss: 1.1168
Epoch 20 - Train Loss: 1

In [46]:
exp2('tractography_fa' ,tractography_fa , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2214 - Val Loss: 4.6349
Epoch 2 - Train Loss: 1.9114 - Val Loss: 207.3789
Epoch 3 - Train Loss: 1.8235 - Val Loss: 3.3179
Epoch 4 - Train Loss: 1.8716 - Val Loss: 4.1409
Epoch 5 - Train Loss: 1.4673 - Val Loss: 1.8226
Epoch 6 - Train Loss: 1.5582 - Val Loss: 11645.1425
Epoch 7 - Train Loss: 1.6105 - Val Loss: 1.2948
Epoch 8 - Train Loss: 1.5026 - Val Loss: 1.2458
Epoch 9 - Train Loss: 1.2848 - Val Loss: 1.2400
Epoch 10 - Train Loss: 1.4896 - Val Loss: 1.2480
Epoch 11 - Train Loss: 1.4934 - Val Loss: 195301.5215
Epoch 12 - Train Loss: 1.6383 - Val Loss: 1.2069
Epoch 13 - Train Loss: 1.3773 - Val Loss: 1.1913
Epoch 14 - Train Loss: 1.2866 - Val Loss: 1.6113
Epoch 15 - Train Loss: 1.1930 - Val Loss: 1.1928
Epoch 16 - Train Loss: 1.2264 - Val Loss: 1.1928
Epoch 17 - Train Loss: 1.2123 - Val Loss: 1.1763
Epoch 18 - Train Loss: 1.1551 - Val Loss: 1.1714
Epoch 19 - Train Loss: 1.2136 - Val Loss: 1.1727
Epoch 20 - Tr

In [47]:
exp2('tractography_icvf' ,tractography_icvf , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2880 - Val Loss: 5.6251
Epoch 2 - Train Loss: 1.9366 - Val Loss: 55.5238
Epoch 3 - Train Loss: 1.8380 - Val Loss: 1.4773
Epoch 4 - Train Loss: 1.5669 - Val Loss: 1.3601
Epoch 5 - Train Loss: 1.5829 - Val Loss: 1.3652
Epoch 6 - Train Loss: 1.6884 - Val Loss: 1.2955
Epoch 7 - Train Loss: 1.3371 - Val Loss: 1.2772
Epoch 8 - Train Loss: 1.5114 - Val Loss: 1.2280
Epoch 9 - Train Loss: 1.3038 - Val Loss: 1.2234
Epoch 10 - Train Loss: 1.5087 - Val Loss: 1.2201
Epoch 11 - Train Loss: 1.4849 - Val Loss: 1.2014
Epoch 12 - Train Loss: 1.2831 - Val Loss: 1.1770
Epoch 13 - Train Loss: 1.3136 - Val Loss: 1.1876
Epoch 14 - Train Loss: 1.3374 - Val Loss: 1.1745
Epoch 15 - Train Loss: 1.3382 - Val Loss: 1.1689
Epoch 16 - Train Loss: 1.2294 - Val Loss: 1.1491
Epoch 17 - Train Loss: 1.2978 - Val Loss: 1.1543
Epoch 18 - Train Loss: 1.1390 - Val Loss: 1.1457
Epoch 19 - Train Loss: 1.1938 - Val Loss: 1.1461
Epoch 20 - Train Loss: 

In [48]:
exp2('tractography_isovf' ,tractography_isovf , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2488 - Val Loss: 1.3704
Epoch 2 - Train Loss: 1.9868 - Val Loss: 1.4990
Epoch 3 - Train Loss: 1.9779 - Val Loss: 1.2861
Epoch 4 - Train Loss: 1.5447 - Val Loss: 1.2531
Epoch 5 - Train Loss: 1.6893 - Val Loss: 1.5124
Epoch 6 - Train Loss: 1.3365 - Val Loss: 1.2359
Epoch 7 - Train Loss: 1.5462 - Val Loss: 1.1927
Epoch 8 - Train Loss: 1.3012 - Val Loss: 1.1608
Epoch 9 - Train Loss: 1.3922 - Val Loss: 1.1473
Epoch 10 - Train Loss: 1.3111 - Val Loss: 1.1334
Epoch 11 - Train Loss: 1.5163 - Val Loss: 1.1277
Epoch 12 - Train Loss: 1.4786 - Val Loss: 1.1277
Epoch 13 - Train Loss: 1.2845 - Val Loss: 1.1072
Epoch 14 - Train Loss: 1.4296 - Val Loss: 1.0940
Epoch 15 - Train Loss: 1.1727 - Val Loss: 1.0944
Epoch 16 - Train Loss: 1.2714 - Val Loss: 1.0930
Epoch 17 - Train Loss: 1.1308 - Val Loss: 1.0819
Epoch 18 - Train Loss: 1.0898 - Val Loss: 1.0774
Epoch 19 - Train Loss: 1.2272 - Val Loss: 1.0742
Epoch 20 - Train Loss: 1

In [49]:
exp2('tractography_md' ,tractography_md , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 1.7746 - Val Loss: 1.3180
Epoch 2 - Train Loss: 1.7954 - Val Loss: 1.2918
Epoch 3 - Train Loss: 1.8149 - Val Loss: 1.7074
Epoch 4 - Train Loss: 1.6760 - Val Loss: 16.5194
Epoch 5 - Train Loss: 1.7030 - Val Loss: 1.3218
Epoch 6 - Train Loss: 1.5139 - Val Loss: 3.7822
Epoch 7 - Train Loss: 1.3599 - Val Loss: 1.3628
Epoch 8 - Train Loss: 1.2263 - Val Loss: 1.2098
Epoch 9 - Train Loss: 1.4150 - Val Loss: 1.2022
Epoch 10 - Train Loss: 1.4675 - Val Loss: 1.2310
Epoch 11 - Train Loss: 1.3065 - Val Loss: 1.1789
Epoch 12 - Train Loss: 1.2416 - Val Loss: 1.1534
Epoch 13 - Train Loss: 1.3158 - Val Loss: 1.1695
Epoch 14 - Train Loss: 1.1778 - Val Loss: 1.1624
Epoch 15 - Train Loss: 1.1809 - Val Loss: 1.1960
Epoch 16 - Train Loss: 1.2207 - Val Loss: 1.1227
Epoch 17 - Train Loss: 1.2662 - Val Loss: 1.1465
Epoch 18 - Train Loss: 1.1599 - Val Loss: 2.1368
Epoch 19 - Train Loss: 1.1137 - Val Loss: 1.1252
Epoch 20 - Train Loss: 

In [50]:
exp2('tractography_od' ,tractography_od , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1451 - Val Loss: 4.0673
Epoch 2 - Train Loss: 2.0446 - Val Loss: 2.8514
Epoch 3 - Train Loss: 1.8497 - Val Loss: 9.4759
Epoch 4 - Train Loss: 2.0545 - Val Loss: 1.3414
Epoch 5 - Train Loss: 1.5890 - Val Loss: 1.2682
Epoch 6 - Train Loss: 1.6508 - Val Loss: 1.2601
Epoch 7 - Train Loss: 1.6260 - Val Loss: 1.2389
Epoch 8 - Train Loss: 1.4319 - Val Loss: 1.2387
Epoch 9 - Train Loss: 1.6565 - Val Loss: 1.2347
Epoch 10 - Train Loss: 1.2918 - Val Loss: 1.2078
Epoch 11 - Train Loss: 1.2301 - Val Loss: 1.1836
Epoch 12 - Train Loss: 1.4124 - Val Loss: 119.3145
Epoch 13 - Train Loss: 1.4609 - Val Loss: 1.1883
Epoch 14 - Train Loss: 1.1194 - Val Loss: 1.2579
Epoch 15 - Train Loss: 1.3423 - Val Loss: 1.1709
Epoch 16 - Train Loss: 1.2703 - Val Loss: 1.1433
Epoch 17 - Train Loss: 1.3362 - Val Loss: 1.1579
Epoch 18 - Train Loss: 1.0573 - Val Loss: 1.1393
Epoch 19 - Train Loss: 1.1461 - Val Loss: 1.1379
Epoch 20 - Train Loss:

In [51]:
exp2('tractography_rd' ,tractography_rd , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.2794 - Val Loss: 1.3423
Epoch 2 - Train Loss: 2.1313 - Val Loss: 1.4034
Epoch 3 - Train Loss: 1.9611 - Val Loss: 13.6951
Epoch 4 - Train Loss: 1.9337 - Val Loss: 21.0885
Epoch 5 - Train Loss: 1.7158 - Val Loss: 1.3390
Epoch 6 - Train Loss: 1.5534 - Val Loss: 1.5498
Epoch 7 - Train Loss: 1.5665 - Val Loss: 2.0115
Epoch 8 - Train Loss: 1.5179 - Val Loss: 1.9790
Epoch 9 - Train Loss: 1.4116 - Val Loss: 1.2468
Epoch 10 - Train Loss: 1.3484 - Val Loss: 1.2431
Epoch 11 - Train Loss: 1.4792 - Val Loss: 1.2532
Epoch 12 - Train Loss: 1.1643 - Val Loss: 1.3366
Epoch 13 - Train Loss: 1.2218 - Val Loss: 1.2892
Epoch 14 - Train Loss: 1.3090 - Val Loss: 1.1924
Epoch 15 - Train Loss: 1.2859 - Val Loss: 1.1928
Epoch 16 - Train Loss: 1.2143 - Val Loss: 1.8259
Epoch 17 - Train Loss: 1.0834 - Val Loss: 1.2608
Epoch 18 - Train Loss: 1.1576 - Val Loss: 1.1765
Epoch 19 - Train Loss: 1.1245 - Val Loss: 1.2426
Epoch 20 - Train Loss:

In [52]:
exp2('tractography_fss' ,tractography_fss  , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1024 - Val Loss: 40.2805
Epoch 2 - Train Loss: 2.1301 - Val Loss: 160.6720
Epoch 3 - Train Loss: 1.9593 - Val Loss: 1.3246
Epoch 4 - Train Loss: 1.7851 - Val Loss: 1.2663
Epoch 5 - Train Loss: 1.7669 - Val Loss: 328.6005
Epoch 6 - Train Loss: 1.7974 - Val Loss: 1.2308
Epoch 7 - Train Loss: 1.4679 - Val Loss: 1.1701
Epoch 8 - Train Loss: 1.4016 - Val Loss: 1.2331
Epoch 9 - Train Loss: 1.4044 - Val Loss: 1.1503
Epoch 10 - Train Loss: 1.2886 - Val Loss: 1.1535
Epoch 11 - Train Loss: 1.3504 - Val Loss: 1.1335
Epoch 12 - Train Loss: 1.3902 - Val Loss: 1.1244
Epoch 13 - Train Loss: 1.5202 - Val Loss: 1.1143
Epoch 14 - Train Loss: 1.3662 - Val Loss: 1.1087
Epoch 15 - Train Loss: 1.2735 - Val Loss: 1.1059
Epoch 16 - Train Loss: 1.2441 - Val Loss: 1.1009
Epoch 17 - Train Loss: 1.2318 - Val Loss: 1.0935
Epoch 18 - Train Loss: 1.1168 - Val Loss: 1.0851
Epoch 19 - Train Loss: 1.2616 - Val Loss: 1.0853
Epoch 20 - Train Lo

In [53]:
exp2('tractography_fssl' ,tractography_fssl , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.3857 - Val Loss: 80.0410
Epoch 2 - Train Loss: 2.2180 - Val Loss: 10.6871
Epoch 3 - Train Loss: 1.9106 - Val Loss: 1.4358
Epoch 4 - Train Loss: 1.7624 - Val Loss: 1.4286
Epoch 5 - Train Loss: 1.7513 - Val Loss: 1.4048
Epoch 6 - Train Loss: 1.6567 - Val Loss: 1.3168
Epoch 7 - Train Loss: 1.6548 - Val Loss: 1.2809
Epoch 8 - Train Loss: 1.5707 - Val Loss: 1.2538
Epoch 9 - Train Loss: 1.4901 - Val Loss: 1.2536
Epoch 10 - Train Loss: 1.4500 - Val Loss: 1.2322
Epoch 11 - Train Loss: 1.3636 - Val Loss: 1.2178
Epoch 12 - Train Loss: 1.3635 - Val Loss: 1.2065
Epoch 13 - Train Loss: 1.4570 - Val Loss: 1.2017
Epoch 14 - Train Loss: 1.2106 - Val Loss: 1.1938
Epoch 15 - Train Loss: 1.2758 - Val Loss: 1.1893
Epoch 16 - Train Loss: 1.1852 - Val Loss: 1.1888
Epoch 17 - Train Loss: 1.4319 - Val Loss: 1.1796
Epoch 18 - Train Loss: 1.4051 - Val Loss: 1.1734
Epoch 19 - Train Loss: 1.2062 - Val Loss: 1.1664
Epoch 20 - Train Loss:

In [54]:
exp2('f_gamb' ,f_gamb , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1890 - Val Loss: 2.4688
Epoch 2 - Train Loss: 1.9795 - Val Loss: 1.6197
Epoch 3 - Train Loss: 1.8652 - Val Loss: 1.4707
Epoch 4 - Train Loss: 1.5655 - Val Loss: 1.3302
Epoch 5 - Train Loss: 1.6914 - Val Loss: 1.2825
Epoch 6 - Train Loss: 1.4353 - Val Loss: 1.2815
Epoch 7 - Train Loss: 1.5883 - Val Loss: 1.2746
Epoch 8 - Train Loss: 1.4528 - Val Loss: 1.2450
Epoch 9 - Train Loss: 1.6270 - Val Loss: 1.2711
Epoch 10 - Train Loss: 1.4698 - Val Loss: 1.2193
Epoch 11 - Train Loss: 1.4406 - Val Loss: 1.2094
Epoch 12 - Train Loss: 1.3205 - Val Loss: 1.1989
Epoch 13 - Train Loss: 1.3734 - Val Loss: 1.1903
Epoch 14 - Train Loss: 1.3384 - Val Loss: 1.1942
Epoch 15 - Train Loss: 1.2426 - Val Loss: 1.1865
Epoch 16 - Train Loss: 1.2184 - Val Loss: 1.1764
Epoch 17 - Train Loss: 1.1330 - Val Loss: 1.1791
Epoch 18 - Train Loss: 1.3200 - Val Loss: 1.1704
Epoch 19 - Train Loss: 1.1840 - Val Loss: 1.1635
Epoch 20 - Train Loss: 1

In [55]:
exp2('f_lang' ,f_lang , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1337 - Val Loss: 2.4400
Epoch 2 - Train Loss: 1.9990 - Val Loss: 9.6371
Epoch 3 - Train Loss: 1.8910 - Val Loss: 1.5167
Epoch 4 - Train Loss: 1.8895 - Val Loss: 1.3986
Epoch 5 - Train Loss: 1.5709 - Val Loss: 1.3865
Epoch 6 - Train Loss: 1.4700 - Val Loss: 16.9014
Epoch 7 - Train Loss: 1.7243 - Val Loss: 1.3382
Epoch 8 - Train Loss: 1.4503 - Val Loss: 1.2875
Epoch 9 - Train Loss: 1.3510 - Val Loss: 1.2570
Epoch 10 - Train Loss: 1.2558 - Val Loss: 1.2374
Epoch 11 - Train Loss: 1.3826 - Val Loss: 1.2186
Epoch 12 - Train Loss: 1.3766 - Val Loss: 1.2054
Epoch 13 - Train Loss: 1.4188 - Val Loss: 1.2173
Epoch 14 - Train Loss: 1.5467 - Val Loss: 1.2034
Epoch 15 - Train Loss: 1.2269 - Val Loss: 1.1861
Epoch 16 - Train Loss: 1.2049 - Val Loss: 1.1860
Epoch 17 - Train Loss: 1.1275 - Val Loss: 1.1684
Epoch 18 - Train Loss: 1.2037 - Val Loss: 1.1683
Epoch 19 - Train Loss: 1.0620 - Val Loss: 1.1492
Epoch 20 - Train Loss: 

In [56]:
exp2('f_motor' ,f_motor , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1419 - Val Loss: 1.5121
Epoch 2 - Train Loss: 1.8931 - Val Loss: 1.4214
Epoch 3 - Train Loss: 1.9772 - Val Loss: 1.3538
Epoch 4 - Train Loss: 1.7762 - Val Loss: 1.3389
Epoch 5 - Train Loss: 1.6018 - Val Loss: 1.2820
Epoch 6 - Train Loss: 1.5030 - Val Loss: 1.2794
Epoch 7 - Train Loss: 1.5813 - Val Loss: 1.2530
Epoch 8 - Train Loss: 1.5944 - Val Loss: 1.2244
Epoch 9 - Train Loss: 1.3833 - Val Loss: 1.2082
Epoch 10 - Train Loss: 1.4237 - Val Loss: 1.1964
Epoch 11 - Train Loss: 1.3498 - Val Loss: 1.1928
Epoch 12 - Train Loss: 1.3810 - Val Loss: 1.1832
Epoch 13 - Train Loss: 1.3527 - Val Loss: 1.1794
Epoch 14 - Train Loss: 1.1831 - Val Loss: 1.1731
Epoch 15 - Train Loss: 1.1882 - Val Loss: 1.1660
Epoch 16 - Train Loss: 1.3056 - Val Loss: 1.1602
Epoch 17 - Train Loss: 1.1838 - Val Loss: 1.1556
Epoch 18 - Train Loss: 1.3849 - Val Loss: 1.1581
Epoch 19 - Train Loss: 1.0333 - Val Loss: 1.1535
Epoch 20 - Train Loss: 1

In [57]:
exp2('f_rs' ,f_rs , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.0578 - Val Loss: 1.5518
Epoch 2 - Train Loss: 1.8879 - Val Loss: 1.4397
Epoch 3 - Train Loss: 1.7609 - Val Loss: 1.5310
Epoch 4 - Train Loss: 1.7458 - Val Loss: 1.3553
Epoch 5 - Train Loss: 1.6346 - Val Loss: 1.3461
Epoch 6 - Train Loss: 1.5780 - Val Loss: 1.2886
Epoch 7 - Train Loss: 1.5662 - Val Loss: 1.2866
Epoch 8 - Train Loss: 1.4176 - Val Loss: 1.2371
Epoch 9 - Train Loss: 1.4354 - Val Loss: 1.2172
Epoch 10 - Train Loss: 1.3964 - Val Loss: 1.2009
Epoch 11 - Train Loss: 1.2731 - Val Loss: 1.1830
Epoch 12 - Train Loss: 1.2550 - Val Loss: 1.1704
Epoch 13 - Train Loss: 1.6473 - Val Loss: 1.1696
Epoch 14 - Train Loss: 1.2759 - Val Loss: 1.1441
Epoch 15 - Train Loss: 1.3977 - Val Loss: 1.1281
Epoch 16 - Train Loss: 1.3454 - Val Loss: 1.1251
Epoch 17 - Train Loss: 1.0848 - Val Loss: 1.1314
Epoch 18 - Train Loss: 1.2213 - Val Loss: 1.1181
Epoch 19 - Train Loss: 1.1692 - Val Loss: 1.1057
Epoch 20 - Train Loss: 1

In [58]:
exp2('f_social' ,f_social , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.1938 - Val Loss: 10.0236
Epoch 2 - Train Loss: 2.0885 - Val Loss: 1.5306
Epoch 3 - Train Loss: 1.7602 - Val Loss: 1.4679
Epoch 4 - Train Loss: 2.1946 - Val Loss: 10.0250
Epoch 5 - Train Loss: 1.9410 - Val Loss: 1.3727
Epoch 6 - Train Loss: 1.6369 - Val Loss: 1.3121
Epoch 7 - Train Loss: 1.4697 - Val Loss: 1.2898
Epoch 8 - Train Loss: 1.5877 - Val Loss: 1.2614
Epoch 9 - Train Loss: 1.5194 - Val Loss: 1.2669
Epoch 10 - Train Loss: 1.4415 - Val Loss: 1.2477
Epoch 11 - Train Loss: 1.2509 - Val Loss: 1.2270
Epoch 12 - Train Loss: 1.3577 - Val Loss: 48.7889
Epoch 13 - Train Loss: 1.2830 - Val Loss: 1.2176
Epoch 14 - Train Loss: 1.2525 - Val Loss: 1.2023
Epoch 15 - Train Loss: 1.2259 - Val Loss: 1.1885
Epoch 16 - Train Loss: 1.0861 - Val Loss: 1.1789
Epoch 17 - Train Loss: 1.2208 - Val Loss: 1.1830
Epoch 18 - Train Loss: 1.1152 - Val Loss: 1.1629
Epoch 19 - Train Loss: 1.3337 - Val Loss: 1.1634
Epoch 20 - Train Loss

In [59]:
exp2('f_wm' ,f_wm , 50, 10)

iter  1
(602, 79800) (151, 79800) (602, 59) (151, 59)
Epoch 1 - Train Loss: 2.0871 - Val Loss: 1.6073
Epoch 2 - Train Loss: 1.9289 - Val Loss: 1.5734
Epoch 3 - Train Loss: 1.9147 - Val Loss: 1.3612
Epoch 4 - Train Loss: 1.7083 - Val Loss: 1.2438
Epoch 5 - Train Loss: 1.7936 - Val Loss: 1.2251
Epoch 6 - Train Loss: 1.8693 - Val Loss: 1.2276
Epoch 7 - Train Loss: 1.8019 - Val Loss: 1.2092
Epoch 8 - Train Loss: 1.5757 - Val Loss: 1.2457
Epoch 9 - Train Loss: 1.1523 - Val Loss: 1.1598
Epoch 10 - Train Loss: 1.3300 - Val Loss: 1.1399
Epoch 11 - Train Loss: 1.4100 - Val Loss: 1.1538
Epoch 12 - Train Loss: 1.4652 - Val Loss: 1.1263
Epoch 13 - Train Loss: 1.1508 - Val Loss: 1.1137
Epoch 14 - Train Loss: 1.1995 - Val Loss: 1.1060
Epoch 15 - Train Loss: 1.1082 - Val Loss: 1.0983
Epoch 16 - Train Loss: 1.3437 - Val Loss: 1.1004
Epoch 17 - Train Loss: 1.0700 - Val Loss: 1.0871
Epoch 18 - Train Loss: 1.3449 - Val Loss: 1.0881
Epoch 19 - Train Loss: 1.2322 - Val Loss: 1.0823
Epoch 20 - Train Loss: 1